<div style="text-align: center;">
    <font size="7"><b>Preparing Salaries Data</b></font>
</div>

<div class="c">
This notebook is for preparing data for Analysis and Prediction. <br>
Classical preparation steps are taken here: <br>
    - Uniformization of terminology/ columns/ values/ units of measurement. <br>
    - Handling missing and mistyped values. <br>
    - Detecting outliers. <br>
    - Deriving new variables. <br>
    - Reporting the data quality metrics.<br>
<br>
A few words about the sources:<br>
Three main sources were used for salary data; which repeated the survey yearly.<br>
    1.) Kaggle.com, a central professional social-network for anything that is data-related. <br>
&nbsp;&nbsp;&nbsp;&nbsp The data from Kaggle is enormous, and is filled out by individualy coming from a wide variaty of tech discciplines.<br>
    2.) AI-Jobs.net, which is also centered around data-professionals, but not exclusively. This is a private recruitment & job-listing company.<br>
    3.) Germany IT-Survey, a germany-specific survey targeting tech professionals. This is quite welcome, as I'm specifically interested in germany's IT landscape.
</div>

<font size="6"><b>Table of Content:</b></font>

In [5]:
import sys
sys.path.append('../')
from scripts.tableofcontent_generator import generate_toc, generate_toc_withanchors
notebook_path = '../notebooks/Salaries_Preparation.ipynb'

In [6]:
toc_content = generate_toc(notebook_path)
print(toc_content)

     1 Import libraries & data, general settings
         1.1 Styles
         1.2 Basic standardization
         1.3 Downcasting data types for better memory usage
             1.3.1 Kaggle
             1.3.2 Germany IT survey
             1.3.3 AI-Jobs.net
     2 Comprehending the data. Uniformization.
             2.0.1 Importing the Clean_Salary function
         2.1 AI-Jobs.net
             2.1.1 Exchange rates
         2.2 Germany IT survey
             2.2.1 Year: 2018
             2.2.2 Year: 2019
             2.2.3 Year: 2020
                 2.2.3.1 Renaming the columns
                 2.2.3.2 Checking cleanliness
                 2.2.3.3 Converting to USD
             2.2.4 Year: 2021
                 2.2.4.1 checking cleanliness
             2.2.5 Year: 2022
             2.2.6 Year: 2023
         2.3 Kaggle
             2.3.1 Year: 2019
             2.3.2 Year: 2020
             2.3.3 Year: 2021
             2.3.4 Year: 2022
         2.4 Final checking uniformity
         2

# Import libraries & data, general settings

In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
from datetime import datetime
import re
from IPython.display import HTML, display

import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp as mc
from scipy.stats import levene, shapiro

# If the notebook is opened from the "notebooks" folder, we need to append the main directory to the "python path" so it sees all subfolders.
import sys
sys.path.append('../')

In [9]:
df_it18_ini = pd.read_csv('../data/raw/IT_Salary_Survey_EU_2018.csv', low_memory=False)
df_it19_ini = pd.read_csv('../data/raw/IT_Salary_Survey_EU_2019.csv', low_memory=False)
df_it20_ini = pd.read_csv('../data/raw/IT_Salary_Survey_EU_2020.csv', low_memory=False)
df_it21_ini = pd.read_csv('../data/raw/IT_Salary_Survey_EU_2021.csv', low_memory=False)
df_it22_ini = pd.read_csv('../data/raw/IT_Salary_Survey_EU_2022.csv', low_memory=False)
df_it23_ini = pd.read_csv('../data/raw/IT_Salary_Survey_EU_2023.csv', low_memory=False)

df_k19_ini = pd.read_csv('../data/raw/kaggle_survey_2019_responses.csv', low_memory=False)
df_k20_ini = pd.read_csv('../data/raw/kaggle_survey_2020_responses.csv', low_memory=False)
df_k21_ini = pd.read_csv('../data/raw/kaggle_survey_2021_responses.csv', low_memory=False)
df_k22_ini = pd.read_csv('../data/raw/kaggle_survey_2022_responses.csv', low_memory=False)

df_ai_ini = pd.read_csv('../data/raw/ai-jobsnet_salaries_2024.csv', low_memory=False)

In [10]:
country_salary_stats = pd.read_csv('../data/world_economic_indices/country_salary_stats.csv', sep=';', low_memory=False)

In [11]:
dfs_ini = [
    df_it18_ini,
    df_it19_ini,
    df_it20_ini,
    df_it21_ini,
    df_it22_ini,
    df_it23_ini,
    df_k19_ini,
    df_k20_ini,
    df_k21_ini,
    df_k22_ini,
    df_ai_ini,
    country_salary_stats]

In [12]:
len_it18_ini = len(df_it18_ini)
len_it19_ini = len(df_it19_ini)
len_it20_ini = len(df_it20_ini)
len_it21_ini = len(df_it21_ini)
len_it22_ini = len(df_it22_ini)
len_it23_ini = len(df_it23_ini)

len_k19_ini = len(df_k19_ini)
len_k20_ini = len(df_k20_ini)
len_k21_ini = len(df_k21_ini)
len_k22_ini = len(df_k22_ini)

len_ai_ini = len(df_ai_ini)

## Styles

In [14]:
# General Display settings

# Column display is supressed by default
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

#changing the display format
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Plotting format
#print(plt.style.available)
plt.style.use('seaborn-v0_8-whitegrid')

In [15]:
style_light_theme = """
<style>
h1 {
    background-color: #0e2e3b;
    color: white;
    font-size: 40px !important;
    font-weight: 700 !important;
    padding: 10px;
}
h2 {
    background-color: #07447E;
    color: white;
    font-size: 35px !important;
    font-weight: 700 !important;
    padding: 10px;
}
h3 {
    background-color: #047c98;
    color: white;
    font-size: 30px !important;
    font-weight: 700 !important;
    padding: 10px;
}
h4 {
    background-color: #0AB89E;
    color: white;
    font-size: 25px !important;
    font-weight: 700 !important;
    padding: 5px;
}
/* ----------------------------------  Conclusion class */
.c {
    background-color: #f7fe9a;
    color: black;
    padding: 10px 10px 10px 20px;        /* Top, Right, Bottom, Left */
    font-size: 16px;
    font-style: italic;
}
/* ----------------------------------  Note class */
.note {
    background-color: #f4fcc0;
    color: black;
    padding: 2px 10px 2px 20px;        /* Top, Right, Bottom, Left */
    font-size: 14px;
    font-style: italic;
}
</style>
"""

In [16]:
#display(HTML(style_dark_theme))
display(HTML(style_light_theme))

## Basic standardization

In [18]:
# Column names to lowercase
for df in dfs_ini:
    df.columns = df.columns.str.lower()

## Values to lowercase
#for i in range(len(dfs)):
#    dfs[i] = dfs[i].applymap(lambda x: x.lower() if isinstance(x, str) else x)

# Whitespaces in column names
for df in dfs_ini:
    df.columns = df.columns.str.replace(' ', '_')

In [19]:
df_it20_ini.head(2)

,timestamp,age,gender,city,position_,total_years_of_experience,years_of_experience_in_germany,seniority_level,your_main_technology_/_programming_language,other_technologies/programming_languages_you_use_often,yearly_brutto_salary_(without_bonus_and_stocks)_in_eur,yearly_bonus_+_stocks_in_eur,annual_brutto_salary_(without_bonus_and_stocks)_one_year_ago._only_answer_if_staying_in_the_same_country,annual_bonus+stocks_one_year_ago._only_answer_if_staying_in_same_country,number_of_vacation_days,employment_status,сontract_duration,main_language_at_work,company_size,company_type,have_you_lost_your_job_due_to_the_coronavirus_outbreak?,"have_you_been_forced_to_have_a_shorter_working_week_(kurzarbeit)?_if_yes,_how_many_hours_per_week","have_you_received_additional_monetary_support_from_your_employer_due_to_work_from_home?_if_yes,_how_much_in_2020_in_eur"
0,24/11/2020 11:14:15,26.00,Male,Munich,Software Engineer,5,3,Senior,TypeScript,"Kotlin, Javascript / Typescript",80000.00,5000,75000.00,10000,30,Full-time employee,Unlimited contract,English,51-100,Product,No,NaN,NaN
1,24/11/2020 11:14:16,26.00,Male,Berlin,Backend Developer,7,4,Senior,Ruby,NaN,80000.00,NaN,82000.00,5000,28,Full-time employee,Unlimited contract,English,101-1000,Product,No,NaN,NaN


In [20]:
# # Every datapoints to lowercase
# 
# df_it18_ini =df_it18_ini.applymap(lambda x: x.lower() if isinstance(x, str) else x)
# df_it19_ini =df_it19_ini.applymap(lambda x: x.lower() if isinstance(x, str) else x)
# df_it20_ini =df_it20_ini.applymap(lambda x: x.lower() if isinstance(x, str) else x)
# df_it21_ini =df_it21_ini.applymap(lambda x: x.lower() if isinstance(x, str) else x)
# df_it22_ini =df_it22_ini.applymap(lambda x: x.lower() if isinstance(x, str) else x)
# df_it23_ini =df_it23_ini.applymap(lambda x: x.lower() if isinstance(x, str) else x)
# df_k19_ini = df_k19_ini.applymap(lambda x: x.lower() if isinstance(x, str) else x)
# df_k20_ini = df_k20_ini.applymap(lambda x: x.lower() if isinstance(x, str) else x)
# df_k21_ini = df_k21_ini.applymap(lambda x: x.lower() if isinstance(x, str) else x)
# df_k22_ini = df_k22_ini.applymap(lambda x: x.lower() if isinstance(x, str) else x)
# df_ai_ini =df_ai_ini.applymap(lambda x: x.lower() if isinstance(x, str) else x)
# country_salary_stats = country_salary_stats.applymap(lambda x: x.lower() if isinstance(x, str) else x)

In [21]:
for df in dfs_ini:
    str_columns = df.select_dtypes(include=['object']).columns
    df[str_columns] = df[str_columns].map(lambda x: x.lower() if isinstance(x, str) else x)

## Downcasting data types for better memory usage

In [23]:
from scripts.memory_summary import memory_summary

In [24]:
?? memory_summary

Signature:  memory_summary(df, num_rows=None)
Source:   
def memory_summary(df, num_rows=None):
    """ A simple function to list out the memory usage of each column.
    Inputs: (df, an integer to control how many columns should be printed)"""
    # Calculate Total memory usage:
    print("Total memory Usage:")
    print(f"\t{round(df.memory_usage(deep=True).sum() / (1024 * 1024), 2)} MB")
    
    # Calculate memory usage for each column
    memory_usage_per_column = {column: round(df[column].memory_usage(deep=True) / (1024 * 1024), 2) for column in df.columns}
    # Sort columns by memory usage in descending order
    sorted_columns_by_memory = sorted(memory_usage_per_column.items(), key=lambda x: x[1], reverse=True)
    
    # Print detailed memory usage in descending order
    print(f"Detailed memory Usage [{len(sorted_columns_by_memory)} columns] (descending order):")
    if num_rows is None:
        num_rows = len(sorted_columns_by_memory)
    for i in range(min(num_rows, len(so

### Kaggle 

In [26]:
memory_summary(df_k21_ini, 10)

Total memory Usage:
	336.53 MB
Detailed memory Usage [369 columns] (descending order):
	q4: 2.47 MB
	q41: 2.11 MB
	q23: 1.96 MB
	q11: 1.64 MB
	q24_part_1: 1.55 MB
	q5: 1.54 MB
	q15: 1.52 MB
	q6: 1.46 MB
	q42_part_6: 1.46 MB
	q3: 1.44 MB


In [27]:
df_k21_ini = df_k21_ini.astype('category')

In [28]:
memory_summary(df_k21_ini, 10)

Total memory Usage:
	9.67 MB
Detailed memory Usage [369 columns] (descending order):
	time_from_start_to_finish_(seconds): 0.4 MB
	q1: 0.03 MB
	q2: 0.03 MB
	q3: 0.03 MB
	q4: 0.03 MB
	q5: 0.03 MB
	q6: 0.03 MB
	q7_part_1: 0.03 MB
	q7_part_2: 0.03 MB
	q7_part_3: 0.03 MB


<div class="c">
Categorical conversion made a significant impact on memory usage.<br>
It's logical since the survey contained fixed-choice questions.<br>
I proceed by converting all the remaining Kaggle datasets.
</div>

In [30]:
df_k19_ini = df_k19_ini.astype('category')
df_k20_ini = df_k20_ini.astype('category')
#df_k21_ini = df_k21_ini.astype('category')
df_k22_ini = df_k22_ini.astype('category')

### Germany IT survey 

In [32]:
memory_summary(df_it23_ini, 10)

Total memory Usage:
	1.05 MB
Detailed memory Usage [41 columns] (descending order):
	company_industry: 0.07 MB
	employment_status: 0.05 MB
	position: 0.05 MB
	other_technologies/programming_languages_you_use_often: 0.05 MB
	city: 0.04 MB
	seniority_level: 0.04 MB
	main_technology_/_programming_language: 0.04 MB
	what_languages_do_you_speak_and_use_at_work?: 0.04 MB
	company_size: 0.04 MB
	company_layoffs: 0.04 MB


It already requires so little memory usage, that I do not proceed with downcasting

### AI-Jobs.net 

In [35]:
memory_summary(df_ai_ini, 10)

Total memory Usage:
	6.14 MB
Detailed memory Usage [11 columns] (descending order):
	job_title: 0.99 MB
	salary_currency: 0.79 MB
	experience_level: 0.78 MB
	employment_type: 0.78 MB
	employee_residence: 0.78 MB
	company_location: 0.78 MB
	company_size: 0.76 MB
	work_year: 0.12 MB
	salary: 0.12 MB
	salary_in_usd: 0.12 MB


<div class="c">
It already uses an insignificant amount of memory, but I proceed with downcasting to have consistent variable types.
</div>

In [37]:
df_ai_ini = df_ai_ini.astype('category')

# Comprehending the data. Uniformization. 

<div class="c">
The different surveys store the same data under differently named columns.<br>
Furthermore, even the same source, as it repeats the survey yearly, the format is changed up.<br>
Uniformization requires now <u>a lot of manual work</u>, as there is no common structure.<br>
<br>
Uniformization goal:<br>
    - "salary": Yearly gross salary with bonuses included [int, in USD],<br>
    - "year": Year in which the datapoint was recorded [int],<br>
    - "job_title": The title of the job eg.: "Data Engineer"[str],<br>
    - "job_group": An arbitrary grouping of similar job titles: "Data Engineer" & "Database Engineer" --> "DA" [str],<br>
</div> 

In [40]:
# A simple function to list out the column names and the most frequent values.

def top_5_values_per_column(df):
    # Create an empty DataFrame to store the result
    result_df = pd.DataFrame(columns=['column_name', 'top_5_values'])

    # Iterate through each column
    for column in df.columns:
        # Find the five most common values in the column
        top_5_values = df[column].value_counts().head(5).index.tolist()

        # Create a DataFrame with the current column name and top 5 values
        data_to_append = pd.DataFrame({'column_name': [column], 'top_5_values': [top_5_values]})

        # Concatenate the new DataFrame with the result DataFrame
        result_df = pd.concat([result_df, data_to_append], ignore_index=True)

    return result_df

### Importing the Clean_Salary function

In [42]:
# Reloading a module
import importlib
import sys
import scripts.clean_salary

# Add the parent directory of 'scripts' to the module search path
sys.path.append('../')

# # Reload the module
# importlib.reload(scripts.clean_salary)
# from scripts.clean_salary import clean_salary_0616
# df_report = pd.DataFrame(index=[0])

In [43]:
?? clean_salary_0616

Object ` clean_salary_0616` not found.


## AI-Jobs.net

In [45]:
top_5_values_per_column(df_ai_ini)

,column_name,top_5_values
0,work_year,"[2023, 2024, 2022, 2021, 2020]"
1,experience_level,"[se, mi, en, ex]"
2,employment_type,"[ft, pt, ct, fl]"
3,job_title,"[data engineer, data scientist, data analyst, ..."
4,salary,"[150000, 100000, 130000, 160000, 120000]"
5,salary_currency,"[usd, gbp, eur, inr, cad]"
6,salary_in_usd,"[150000, 100000, 130000, 160000, 140000]"
7,employee_residence,"[us, gb, ca, es, de]"
8,remote_ratio,"[0, 100, 50]"
9,company_location,"[us, gb, ca, es, de]"


In [46]:
# Define column name mappings
column_mappings = {
    'work_year': 'year',
    'experience_level': 'seniority_level',
    'employment_type': 'employment_status',
    'employee_residence': 'country',
    'salary': 'salary_in_currency',
    'salary_in_usd': 'salary'
}

# Rename columns using the mappings
df_ai_u = df_ai_ini.rename(columns=column_mappings)
df_ai_u.head(5)

,year,seniority_level,employment_status,job_title,salary_in_currency,salary_currency,salary,country,remote_ratio,company_location,company_size
0,2024,en,ft,data analyst,20000,usd,20000,ke,100,ke,m
1,2024,se,ft,data analyst,147500,usd,147500,us,0,us,m
2,2024,se,ft,data analyst,85000,usd,85000,us,0,us,m
3,2024,se,ft,data architect,175000,usd,175000,us,0,us,m
4,2024,se,ft,data architect,117000,usd,117000,us,0,us,m


In [47]:
df_ai_u['salary'] = df_ai_u['salary'].apply(clean_salary_0616, df_report=df_report, prefix='df_ai_salary')
print(df_report)

NameError: name 'clean_salary_0616' is not defined

In [ ]:
df_ai_u['salary'].info()

<div class="c">
There are no Null values, so we do not need to drop them.
</div> 

### Exchange rates 

<div class="c">
<b>EUR --> USD exchange ratio calculations </b>.
I'm using the ratios that AI-Jobs.net used, since I can calculate it reversely and reuse those rates.<br>
The rates they used was a fixed rate for the entire year, and aligns with the "Average" exchange rates for that year available online.
</div> 

In [ ]:
# Converting salary and salary_in_currency back to numeric
df_ai_u['salary'] = pd.to_numeric(df_ai_u['salary'], errors='coerce')
df_ai_u['salary_in_currency'] = pd.to_numeric(df_ai_u['salary_in_currency'], errors='coerce')

# Calculating the ratios
df_ai_u['ratio'] = df_ai_u['salary'] / df_ai_u['salary_in_currency']

#changing the display format
pd.set_option('display.float_format', lambda x: '%.6f' % x)

#listing out the calculated ratios. Within each year, they're almost the same, except the rounding error.
df_ai_u[(df_ai_u['salary_currency'] == 'eur') & (df_ai_u['year'] == 2020)].value_counts('ratio').head(10)

<div class="c">
Most probably they <b>rounded</b>  the values after conversion, and this resulted in the slightly different ratios above.<br>
For methodology's sake I calculate the exchange ratio for USD columns and take their mean.
</div> 

In [ ]:
eur2usd_2020 = df_ai_u[(df_ai_u['salary_currency'] == 'eur') & (df_ai_u['year'] == 2020)]['ratio'].mean()
eur2usd_2021 = df_ai_u[(df_ai_u['salary_currency'] == 'eur') & (df_ai_u['year'] == 2021)]['ratio'].mean()
eur2usd_2022 = df_ai_u[(df_ai_u['salary_currency'] == 'eur') & (df_ai_u['year'] == 2022)]['ratio'].mean()
eur2usd_2023 = df_ai_u[(df_ai_u['salary_currency'] == 'eur') & (df_ai_u['year'] == 2023)]['ratio'].mean()
eur2usd_2024 = df_ai_u[(df_ai_u['salary_currency'] == 'eur') & (df_ai_u['year'] == 2024)]['ratio'].mean()

# For other years that are not present in the AI-Jobs.net dataset, I use sources available online
eur2usd_2019 = 1.1199 # Source: https://www.exchangerates.org.uk/EUR-USD-spot-exchange-rates-history-2019.html
eur2usd_2018 = 1.1811 # Source: https://www.exchangerates.org.uk/EUR-USD-spot-exchange-rates-history-2018.html

#changing back the display format
pd.set_option('display.float_format', lambda x: '%.2f' % x)

## Germany IT survey 

### Year: 2018

In [ ]:
top_5_values_per_column(df_it18_ini)

In [ ]:
# Adding year and country
df_it18_ini['year'] = 2018
df_it18_ini['country'] = "de"

# Define column name mappings
column_mappings = {
    'position': 'job_title',
    'years_of_experience': 'experience',
    'your_level': 'seniority_level',
    'current_salary': 'salary_eur',
    'main_language_at_work': 'language_at_work'
}

# Rename columns using the mappings
df_it18_u = df_it18_ini.rename(columns=column_mappings)
df_it18_u.head(2)

In [ ]:
# Cleaning report
df_it18_u['salary_eur'] = df_it18_u['salary_eur'].apply(clean_salary_0616, df_report=df_report, prefix='df_it18_salary')
df_report

In [ ]:
len_it18_salarydrop1 = len(df_it18_u) # For data quality report

# Comverting to numeric
df_it18_u['salary_eur'] = df_it18_u['salary_eur'].apply(pd.to_numeric, errors='raise')
# same appraoch: df_it18_u['salary_eur'] = pd.to_numeric(df_it18_u['salary_eur'], errors='raise')

# Dropping rows where salary is Null
df_it18_u.dropna(subset=['salary_eur'], inplace=True)

# Assigning 'int64' datatype. This serves as a self-check
df_it18_u['salary_eur'] = df_it18_u['salary_eur'].astype('int64')

# Create a new 'salary' column that will represent the salary in USD
df_it18_u['salary'] = df_it18_u['salary_eur'] * eur2usd_2018

len_it18_salarydrop2 = len(df_it18_u) # For data quality report

df_it18_u.head(2)

In [ ]:
df_it18_u[['salary_eur','salary']].info()

### Year: 2019

In [ ]:
top_5_values_per_column(df_it19_ini)

In [ ]:
# Adding year and country
df_it19_ini['year'] = 2019
df_it19_ini['country'] = "de"

# Define column name mappings
column_mappings = {
    'position_(without_seniority)': 'job_title',
    'years_of_experience': 'experience',
    'your_main_technology_/_programming_language': 'skills',
    'yearly_brutto_salary_(without_bonus_and_stocks)': 'base_salary',
    'yearly_bonus': 'bonus',
    'yearly_stocks': 'stocks',
    'yearly_brutto_salary_(without_bonus_and_stocks)_one_year_ago._only_answer_if_staying_in_same_country': 'salary_1y_ago',
    'yearly_bonus_one_year_ago._only_answer_if_staying_in_same_country':'bonus_1y_ago',
    'yearly_stocks_one_year_ago._only_answer_if_staying_in_same_country':'stocks_1y_ago',
    'main_language_at_work': 'language_at_work'
}

# Rename columns using the mappings
df_it19_u = df_it19_ini.rename(columns=column_mappings)

In [ ]:
# Cleaning report

df_it19_u['base_salary'] = df_it19_u['base_salary'].apply(clean_salary_0616, df_report=df_report, prefix='df_it19_u_salary')
df_it19_u['bonus'] = df_it19_u['bonus'].apply(clean_salary_0616, df_report=df_report, prefix='df_it19_u_bonus')
df_it19_u['stocks'] = df_it19_u['stocks'].apply(clean_salary_0616, df_report=df_report, prefix='df_it19_u_stocks')
df_report

<div class=c>
There is a Null value in 'base_salary', that will need to be dropped. The cleaning function itself does not drop Null values, so I drop it separately.<br>
There are hundreds of null values in 'bonus' and 'stocks', I'll convert them to zeroes.
</div>

In [ ]:
# Dropping values

len_it19_salarydrop1 = len(df_it19_u) # For data quality report

df_it19_u = df_it19_u.dropna(subset=['base_salary'])
#df_salary_conversion['base_salary'].fillna(0, inplace=True)
df_it19_u['bonus'] = df_it19_u['bonus'].fillna(0)
df_it19_u['stocks'] = df_it19_u['stocks'].fillna(0)

df_it19_u['base_salary'].apply(pd.to_numeric, errors='coerce')
df_it19_u['bonus'].apply(pd.to_numeric, errors='coerce')
df_it19_u['stocks'].apply(pd.to_numeric, errors='coerce')
df_it19_u[['base_salary','bonus','stocks']] = df_it19_u[['base_salary','bonus','stocks']].astype('int64')

# Create a new 'salary' column by adding the three columns together
df_it19_u['salary_eur'] = df_it19_u['base_salary'] + df_it19_u['bonus'] + df_it19_u['stocks']
df_it19_u['salary'] = df_it19_u['salary_eur'] * eur2usd_2019

len_it19_salarydrop2 = len(df_it19_u) # For data quality report

df_it19_u.head(2)

In [ ]:
df_it19_u[['base_salary','bonus','stocks','salary']].info()

### Year: 2020

In [ ]:
top_5_values_per_column(df_it20_ini)

#### Renaming the columns

In [ ]:
# Adding year and country
df_it20_ini['year'] = 2020
df_it20_ini['country'] = "de"

# Define column name mappings
column_mappings = {
    'position_': 'job_title',
    'total_years_of_experience': 'experience',
    'your_main_technology_/_programming_language': 'skills',
    'other_technologies/programming_languages_you_use_often': 'skills_2',
    'yearly_brutto_salary_(without_bonus_and_stocks)_in_eur': 'base_salary',
    'yearly_bonus_+_stocks_in_eur': 'bonus',
    'annual_brutto_salary_(without_bonus_and_stocks)_one_year_ago._only_answer_if_staying_in_the_same_country': 'salary_1y_ago',
    'annual_bonus+stocks_one_year_ago._only_answer_if_staying_in_same_country': 'bonus_1y_ago',
    'yearly_stocks_one_year_ago._only_answer_if_staying_in_same_country':'stocks_1y_ago',
    'main_language_at_work': 'language_at_work'
}

# Rename columns using the mappings
df_it20_u = df_it20_ini.rename(columns=column_mappings)
df_it20_u.head(2)

#### Checking cleanliness

In [ ]:
# Cleaning report

df_it20_u['base_salary'] = df_it20_u['base_salary'].apply(clean_salary_0616, df_report=df_report, prefix='df_it20_u_salary')
df_it20_u['bonus'] = df_it20_u['bonus'].apply(clean_salary_0616, df_report=df_report, prefix='df_it20_u_bonus')
df_report

#### Converting to USD

In [ ]:
len_it20_salarydrop1 = len(df_it20_u)

df_it20_u = df_it20_u.dropna(subset=['base_salary'])
df_it20_u['bonus'] = df_it20_u['bonus'].fillna(0)

#df_it20_u['bonus'] = df_it20_u['bonus'].str.replace('$', '')
#df_it20_u['bonus'] = df_it20_u['bonus'].str.replace('> ', '')

df_it20_u['base_salary'] = pd.to_numeric(df_it20_u['base_salary'], errors='coerce')
df_it20_u['bonus'] = pd.to_numeric(df_it20_u['bonus'], errors='coerce')

df_it20_u['base_salary'] = df_it20_u['base_salary'].fillna(0)
df_it20_u['bonus'] = df_it20_u['bonus'].fillna(0)

#df_it20_u['base_salary'] = df_it20_u['base_salary'].astype('int64')
#df_it20_u['bonus'] = df_it20_u['bonus'].astype('int64')

# Create a new 'salary' column by adding the columns together
df_it20_u['salary_eur'] = df_it20_u['base_salary'] + df_it20_u['bonus']
df_it20_u['salary'] = df_it20_u['salary_eur'] * eur2usd_2020

len_it20_salarydrop2 = len(df_it20_u)

df_it20_u.head(2)

In [ ]:
df_it20_u[['base_salary','bonus','salary']].info()

### Year: 2021

In [ ]:
top_5_values_per_column(df_it21_ini)

In [ ]:
# In some cases the answer were given only in 'annual_brutto_salary_without_bonus_and_stocks_in_eur', and nothing in 'annual_brutto_salary_with_bonus_and_stocks_in_eur'
# Therefore I take the larger of the two
df_it21_ini['salary_eur'] = df_it21_ini[['annual_brutto_salary_without_bonus_and_stocks_in_eur', 'annual_brutto_salary_with_bonus_and_stocks_in_eur']].max(axis=1)

In [ ]:
df_it21_ini['year'] = 2021
df_it21_ini['country'] = "de"

# Define column name mappings
column_mappings = {
    'position': 'job_title',
    'your_position': 'job_title_2',
    'total_years_of_experience': 'experience',
    'main_technology_/_programming_language': 'skills',
    'other_technologies/programming_languages_you_use_often': 'skills_2',
    'annual_brutto_salary_without_bonus_and_stocks_in_eur': 'base_salary',
    #'annual_brutto_salary_with_bonus_and_stocks_in_eur': 'salary_eur',
    'annual_bonus+stocks_one_year_ago.': 'salary_w_bonus_1y_ago',
    'what_languages_do_you_speak_and_use_at_work?': 'language_at_work'
}

# Rename columns using the mappings
df_it21_u = df_it21_ini.rename(columns=column_mappings)
df_it21_u.head(2)

#### checking cleanliness

In [ ]:
df_it21_u['salary_eur'] = df_it21_u['salary_eur'].apply(clean_salary_0616, df_report=df_report, prefix='df_it21_u_salary')
df_report

In [ ]:
len_it21_salarydrop1 = len(df_it21_u)

# dropping initial Nones, that wouldn't be converted with pd.to_numeric errors='raise'
df_it21_u = df_it21_u.dropna(subset=['salary_eur'])

df_it21_u['salary_eur'].apply(pd.to_numeric, errors='raise')

# dropping Nones that is the result of pd.to_numeric
df_it21_u = df_it21_u.dropna(subset=['salary_eur'])

df_it21_u['salary_eur'] = df_it21_u['salary_eur'].astype('int64') # float64 can contain Nones, which is annoying. The data should be convertible to int64 (The only exception is if the answer was given as a float.)

# Create a new 'salary' column by adding the columns together
df_it21_u['salary'] = df_it21_u['salary_eur'] * eur2usd_2021

len_it21_salarydrop2 = len(df_it21_u)

df_it21_u.head(2)

In [ ]:
df_it21_u[['salary_eur','salary']].info()

### Year: 2022 

In [ ]:
top_5_values_per_column(df_it22_ini)

In [ ]:
# In some cases the answer were given only in 'annual_brutto_salary_without_bonus_and_stocks_in_eur', and nothing in 'annual_brutto_salary_with_bonus_and_stocks_in_eur'
# Therefore I take the larger of the two
df_it22_ini['salary_eur'] = df_it22_ini[['annual_gross_salary_without_bonus_and_stocks_in_eur', 'annual_brutto_salary_with_bonus_and_stocks_in_eur']].max(axis=1)

In [ ]:
df_it22_ini['year'] = 2022
df_it22_ini['country'] = "de"

# Define column name mappings
column_mappings = {
    'position': 'job_title',
    'your_position': 'job_title_2',
    'total_years_of_experience': 'experience',
    'main_technology_/_programming_language': 'skills',
    'other_technologies/programming_languages_you_use_often':'skills_2',
    'your_main_technology':'skills_3',
    'annual_gross_salary_without_bonus_and_stocks_in_eur': 'base_salary',
    'annual_brutto_salary_with_bonus_and_stocks_in_eur': 'salary_w_bonus',
    'annual_bonus+stocks_one_year_ago.':'bonus_1y_ago',
    'what_languages_do_you_speak_and_use_at_work?': 'language_at_work'
}

# Rename columns using the mappings
df_it22_u = df_it22_ini.rename(columns=column_mappings)

In [ ]:
df_it22_u['salary_eur'] = df_it22_u['salary_eur'].apply(clean_salary_0616, df_report=df_report, prefix='df_it22_u_salary_eur')
df_report

In [ ]:
# 
# 
# df_it22_u = df_it22_u.dropna(subset=['salary_eur'])
# df_it22_u['salary_eur'].apply(pd.to_numeric, errors='coerce')
# 
# # Create a new 'salary' column by adding the columns together
# df_it22_u['salary'] = df_it22_u['salary_eur'] * eur2usd_2022
# df_it22_u.head(2)

In [ ]:
len_it22_salarydrop1 = len(df_it22_u)

# dropping initial Nones, that wouldn't be converted with pd.to_numeric errors='raise'
df_it22_u = df_it22_u.dropna(subset=['salary_eur'])
df_it22_u['salary_eur'].apply(pd.to_numeric, errors='raise')

# dropping Nones that is the result of pd.to_numeric
df_it22_u = df_it22_u.dropna(subset=['salary_eur'])

df_it22_u['salary_eur'] = df_it22_u['salary_eur'].astype('int64') # float64 can contain Nones, which is annoying. The data should be convertible to int64 (The only exception is if the answer was given as a float.)

# Create a new 'salary' column by adding the columns together
df_it22_u['salary'] = df_it22_u['salary_eur'] * eur2usd_2022
len_it22_salarydrop2 = len(df_it22_u)

df_it22_u.head(2)

In [ ]:
df_it22_u[['salary_eur','salary']].info()

### Year: 2023

In [ ]:
top_5_values_per_column(df_it23_ini)

In [ ]:
# In some cases the answer were given only in 'annual_brutto_salary_without_bonus_and_stocks_in_eur', and nothing in 'annual_brutto_salary_with_bonus_and_stocks_in_eur'
# Therefore I take the larger of the two
df_it23_ini['salary_eur'] = df_it23_ini[['annual_gross_salary_without_bonus_and_stocks_in_eur', 'annual_gross_salary_with_bonus_and_stocks_in_eur']].max(axis=1)

In [ ]:
df_it23_ini['year'] = 2023
df_it23_ini['country'] = "de"

# Define column name mappings
column_mappings = {
    'position': 'job_title',
    'your_position': 'job_title_2',
    'total_years_of_experience': 'experience',
    'main_technology_/_programming_language': 'skills',
    'other_technologies/programming_languages_you_use_often':'skills_2',
    'your_main_technology':'skills_3',
    'annual_gross_salary_without_bonus_and_stocks_in_eur': 'base_salary',
    'annual_gross_salary_with_bonus_and_stocks_in_eur': 'salary_w_bonus',
    'annual_gross_salary+bonus+stocks_one_year_ago.':'salary_w_bonus_1y_ago',
    'what_languages_do_you_speak_and_use_at_work?': 'language_at_work'
}

# Rename columns using the mappings
df_it23_u = df_it23_ini.rename(columns=column_mappings)

In [ ]:
# Cleaning & report
df_it23_u['salary_eur'] = df_it23_u['salary_eur'].apply(clean_salary_0616, df_report=df_report, prefix='df_it23_u_salary_eur')
df_report

In [ ]:
len_it23_salarydrop1 = len(df_it23_u)

# dropping initial Nones, that wouldn't be converted with pd.to_numeric errors='raise'
df_it23_u = df_it23_u.dropna(subset=['salary_eur'])

df_it23_u['salary_eur'].apply(pd.to_numeric, errors='raise')

# dropping Nones that is the result of pd.to_numeric
df_it23_u = df_it23_u.dropna(subset=['salary_eur'])

df_it23_u['salary_eur'] = df_it23_u['salary_eur'].astype('int64') # float64 can contain Nones, which is annoying. The data should be convertible to int64 (The only exception is if the answer was given as a float.)

# Create a new 'salary' column by adding the columns together
df_it23_u['salary'] = df_it23_u['salary_eur'] * eur2usd_2023

len_it23_salarydrop2 = len(df_it23_u)

df_it23_u.head(2)

In [ ]:
df_it23_u[['salary_eur','salary']].info()

## Kaggle 

In [ ]:
def kaggle_summary(df):
    # Create an empty DataFrame to store the result
    result_df = pd.DataFrame(columns=['column_name', 'second_row_values', 'top_5_values'])
    
    # Iterate through each column
    for column in df.columns:
        # Find the five most common values in the column
        top_5_values = df[column].value_counts().head(5).index.tolist()
        
        # Get the value of the second row for the current column
        second_row_value = df.iloc[0][column]
        
        # Create a DataFrame with the current column name, top 5 values, and second row value
        data_to_append = pd.DataFrame({'column_name': [column],
                                       'second_row_values': [second_row_value],
                                       'top_5_values': [top_5_values]
                                       })
        
        # Concatenate the new DataFrame with the result DataFrame
        result_df = pd.concat([result_df, data_to_append], ignore_index=True)
    
    return result_df

### Year: 2019 

In [ ]:
result_df = kaggle_summary(df_k19_ini)
result_df.to_csv('../results/df_k19_structure.txt', sep='\t', index=True)
result_df.head(10)

In [ ]:
# Select only the relevant columns
df_k19_u1 = df_k19_ini[['q1', 'q3', 'q4', 'q5', 'q5_other_text', 'q6', 'q10', 'q15',]]
# Drop the first row as it's only an elaboration on the questionnaire.
df_k19_u1 = df_k19_u1.drop(df_k19_u1.index[0])

# Define column name mappings
column_mappings = {
    'q1': 'age',
    'q3': 'country',
    'q4': 'education_level',
    'q5': 'job_title',
    'q5_other_text': 'job_title_2',
    'q6': 'company_size',
    'q10': 'salary_range',
    'q15': 'experience'
}

# Rename columns using the mappings
df_k19_u = df_k19_u1.rename(columns=column_mappings)
#df_k19_u = df_k19_u1.drop(df_k19_u1.index[0])
df_k19_u['year'] = 2019
df_k19_u.head(3)

In [ ]:
len_k19_salarydrop1 = len(df_k19_u)

df_k19_u['salary_range'] = df_k19_u['salary_range'].str.replace('$', '', regex=False)
df_k19_u['salary_range'] = df_k19_u['salary_range'].str.replace('> ', '', regex=False)

# Dropping rows where salary is Null
df_k19_u.dropna(subset=['salary_range'], inplace=True)

# Split the salary_range column on the dash ('-') and convert to numeric values
df_k19_u[['lower_salary', 'upper_salary']] = df_k19_u['salary_range'].str.split('-', expand=True)
df_k19_u['lower_salary'] = df_k19_u['lower_salary'].str.replace(',', '').astype(float)
df_k19_u['upper_salary'] = df_k19_u['upper_salary'].str.replace(',', '').astype(float)

df_k19_u['salary'] = df_k19_u[['lower_salary', 'upper_salary']].mean(axis=1)

len_k19_salarydrop2 = len(df_k19_u)
df_k19_u.head()

### Year: 2020 

In [ ]:
result_df = kaggle_summary(df_k20_ini)
result_df.to_csv('../results/df_k20_structure.txt', sep='\t', index=True)
result_df.head(10)

In [ ]:
# Select only the important columns
df_k20_u1 = df_k20_ini[['q1', 'q3', 'q4', 'q5', 'q6', 'q20', 'q24']]

# Drop the first row as it's only an elaboration on the questionnaire.
df_k20_u1 = df_k20_u1.drop(df_k20_u1.index[0])

# Define column name mappings
column_mappings = {
    'q1': 'age',
    'q3': 'country',
    'q4': 'education_level',
    'q5': 'job_title',
    'q6': 'experience',
    'q20': 'company_size',
    'q24': 'salary_range'
}

# Rename columns using the mappings
df_k20_u = df_k20_u1.rename(columns=column_mappings)
df_k20_u['year'] = 2020
df_k20_u.head(2)

In [ ]:
len_k20_salarydrop1 = len(df_k20_u)

df_k20_u['salary_range'] = df_k20_u['salary_range'].str.replace('$', '', regex=False)
df_k20_u['salary_range'] = df_k20_u['salary_range'].str.replace('> ', '', regex=False)

# Dropping rows where salary is Null
df_k20_u.dropna(subset=['salary_range'], inplace=True)

# Split the salary_range column on the dash ('-') and convert to numeric values
df_k20_u[['lower_salary', 'upper_salary']] = df_k20_u['salary_range'].str.split('-', expand=True)
df_k20_u['lower_salary'] = df_k20_u['lower_salary'].str.replace(',', '').astype(float)
df_k20_u['upper_salary'] = df_k20_u['upper_salary'].str.replace(',', '').astype(float)

df_k20_u['salary'] = df_k20_u[['lower_salary', 'upper_salary']].mean(axis=1)

len_k20_salarydrop2 = len(df_k20_u)
df_k20_u.head(2)

### Year: 2021

In [ ]:
result_df = kaggle_summary(df_k21_ini)
result_df.to_csv('../results/df_k21_structure.txt', sep='\t', index=True)
result_df.head(10)

In [ ]:
# Select only the relevant columns
df_k21_u1 = df_k21_ini[['q1', 'q3', 'q4', 'q5', 'q6', 'q20', 'q21', 'q25']]

# Drop the first row as it's only an elaboration on the questionnaire.
df_k21_u1 = df_k21_u1.drop(df_k21_u1.index[0])

# Define column name mappings
column_mappings = {
    'q1': 'age',
    'q3': 'country',
    'q4': 'education_level',
    'q5': 'job_title',
    'q6': 'experience',
    'q20': 'industry',
    'q21': 'company_size',
    'q25': 'salary_range'
}

# Rename columns using the mappings
df_k21_u = df_k21_u1.rename(columns=column_mappings)
df_k21_u['year'] = 2021
df_k21_u.head(2)

In [ ]:
len_k21_salarydrop1 = len(df_k21_u)

df_k21_u['salary_range'] = df_k21_u['salary_range'].str.replace('$', '', regex=False)
df_k21_u['salary_range'] = df_k21_u['salary_range'].str.replace('>', '', regex=False)

# Dropping rows where salary is Null
df_k21_u.dropna(subset=['salary_range'], inplace=True)

# Split the salary_range column on the dash ('-') and convert to numeric values
df_k21_u[['lower_salary', 'upper_salary']] = df_k21_u['salary_range'].str.split('-', expand=True)
df_k21_u['lower_salary'] = df_k21_u['lower_salary'].str.replace(',', '').astype(float)
df_k21_u['upper_salary'] = df_k21_u['upper_salary'].str.replace(',', '').astype(float)

df_k21_u['salary'] = df_k21_u[['lower_salary', 'upper_salary']].mean(axis=1)

len_k21_salarydrop2 = len(df_k21_u)
df_k21_u.head(3)

### Year: 2022 

In [ ]:
result_df = kaggle_summary(df_k22_ini)
result_df.to_csv('../results/df_k22_structure.txt', sep='\t', index=True)
result_df.head(10)

In [ ]:
# Select only the relevant columns
df_k22_u1 = df_k22_ini[['q2', 'q4', 'q5', 'q8', 'q11', 'q23', 'q24', 'q25', 'q29']]

# Drop the first row as it's only an elaboration on the questionnaire
df_k22_u1 = df_k22_u1.drop(df_k22_u1.index[0])

# Define column name mappings
column_mappings = {
    'q2': 'age',
    'q4': 'country',
    'q5': 'are_you_student',
    'q8': 'education_level',
    'q11': 'experience',
    'q23': 'job_title',
    'q24': 'industry',
    'q25': 'company_size',
    'q29': 'salary_range'
}

# Rename columns using the mappings
df_k22_u = df_k22_u1.rename(columns=column_mappings)
df_k22_u['year'] = 2022
df_k22_u.head(3)

In [ ]:
len_k22_salarydrop1 = len(df_k22_u)

df_k22_u['salary_range'] = df_k22_u['salary_range'].str.replace('$', '', regex=False)
df_k22_u['salary_range'] = df_k22_u['salary_range'].str.replace('>', '', regex=False)

# Dropping rows where salary is Null
df_k22_u.dropna(subset=['salary_range'], inplace=True)

# Split the salary_range column on the dash ('-') and convert to numeric values
df_k22_u[['lower_salary', 'upper_salary']] = df_k22_u['salary_range'].str.split('-', expand=True)
df_k22_u['lower_salary'] = df_k22_u['lower_salary'].str.replace(',', '').astype(float)
df_k22_u['upper_salary'] = df_k22_u['upper_salary'].str.replace(',', '').astype(float)

df_k22_u['salary'] = df_k22_u[['lower_salary', 'upper_salary']].mean(axis=1)

len_k22_salarydrop2 = len(df_k22_u)
df_k22_u.head(3)

## Final checking uniformity

In [ ]:
df_ai_u['salary'].info()

In [ ]:
df_it18_u['salary'].info()

In [ ]:
df_it19_u['salary'].info()

In [ ]:
df_it20_u['salary'].info()

In [ ]:
df_it21_u['salary'].info()

In [ ]:
df_it22_u['salary'].info()

In [ ]:
df_it23_u['salary'].info()

In [ ]:
df_k19_u['salary'].info()

In [ ]:
df_k20_u['salary'].info()

In [ ]:
df_k21_u['salary'].info()

In [ ]:
df_k22_u['salary'].info()

<div class="c">
There are no null values in any of the 'salary' columns in any dataframe.<br>
Also, they're all float64, which means that all values are recognized as numbers.<br>

Brief note:<br>
Python's Dtypes in relation to NaN and mixed values:<br>
-> NaN can be contained in everything without the dtype reflecting it.<br>
-> When a column contains mixed values (eg.: string and int/float), it is stored as <b>object</b> dtype.
</div> 

## Final cleaning report

In [ ]:
df_report_t = df_report.T
df_report_t.rename(columns={0: 'Occurrence'}, inplace=True)
df_report_t.to_csv('../results/Cleaning_report.txt', sep='\t', index=True)

In [ ]:
len_it18_u = len(df_it18_u)
len_it19_u = len(df_it19_u)
len_it20_u = len(df_it20_u)
len_it21_u = len(df_it21_u)
len_it22_u = len(df_it22_u)
len_it23_u = len(df_it23_u)

len_k19_u = len(df_k19_u)
len_k20_u = len(df_k20_u)
len_k21_u = len(df_k21_u)
len_k22_u = len(df_k22_u)

len_ai_u = len(df_ai_u)

In [ ]:
print(len_it18_u / len_it18_ini),
print(len_it19_u / len_it19_ini),
print(len_it20_u / len_it20_ini),
print(len_it21_u / len_it21_ini),
print(len_it22_u / len_it22_ini),
print(len_it23_u / len_it23_ini),
print(len_k19_u  / len_k19_ini ),
print(len_k20_u  / len_k20_ini ),
print(len_k21_u  / len_k21_ini ),
print(len_k22_u  / len_k22_ini ),
print(len_ai_u   / len_ai_ini  )

<div class="c">
Conclusion:<br>
Throghout the cleaning process, the 'clean_salary' function was used, which not only cleaned irregularities in data, <br>
but upon encountering those irregularities, a counter increased in a reporting dataframe.<br>
Those counters are exported as a <b> cleaning report </b> to the 'results' folder.
</div> 

# Union of the yearly dataframes

<div class="c">
The survey data have been cleaned and prepared now for merging.<br>
These dataframes may contain many interesting and nuanced questions, but for this project, only the common questions will be kept and merged.<br>
Therefore some columns will need to be dropped.<br>
</div> 

In [ ]:
top_5_values_per_column(df_it23_u)

## Germany IT Survey

In [ ]:
df_it18_u = df_it18_u[['age', 'city', 'job_title', 'seniority_level', 'language_at_work', 'company_size', 'company_type', 'salary', 'year', 'country']]

df_it19_u = df_it19_u[['age', 'city', 'job_title', 'seniority_level', 'experience', 'language_at_work', 'company_size', 'company_type', 'salary', 'year', 'country'
                      , 'skills']]

df_it20_u = df_it20_u[['age', 'city', 'job_title', 'seniority_level', 'experience', 'language_at_work', 'company_size', 'company_type', 'salary', 'year', 'country'
                      , 'skills', 'skills_2', 'employment_status', 'years_of_experience_in_germany']]

df_it21_u = df_it21_u[['city', 'job_title', 'job_title_2', 'seniority_level', 'experience', 'language_at_work', 'company_size', 'salary', 'year', 'country'
                      , 'skills', 'skills_2', 'employment_status', 'years_of_experience_in_germany', 'your_seniority_level']]
                       
df_it22_u = df_it22_u[['city', 'job_title', 'job_title_2', 'seniority_level', 'experience', 'language_at_work', 'company_size', 'salary', 'year', 'country'
                      , 'skills', 'skills_2', 'skills_3', 'employment_status', 'years_of_experience_in_germany', 'your_seniority_level']]                       

df_it23_u = df_it23_u[['city', 'job_title', 'job_title_2', 'seniority_level', 'experience', 'language_at_work', 'company_size', 'salary', 'year', 'country'
                      , 'skills', 'skills_2', 'skills_3', 'employment_status', 'years_of_experience_in_germany', 'your_seniority_level', 'company_industry']]                       

In [ ]:
df_it_uni = pd.concat([df_it18_u, df_it19_u, df_it20_u, df_it21_u, df_it22_u, df_it23_u])  #concat creates a copy
df_it_uni['survey'] = 'it'
df_it_uni = df_it_uni.reset_index(drop=True)
len_it_uni = len(df_it_uni)

In [ ]:
df_it_uni.head(2)

## Kaggle

In [ ]:
df_k_uni = pd.concat([df_k19_u, df_k20_u, df_k21_u, df_k22_u]) #concat creates a copy
df_k_uni['survey'] = 'k'
len_k_uni = len(df_k_uni)

In [ ]:
df_k_uni.head(2)

## AI-Jobs.net

In [ ]:
df_ai_uni = df_ai_u.copy() #For naming convention's sake, I create a copy, since for the previous surveys I needed a concat method, which also created copies.
df_ai_uni['survey'] = 'ai'
len_ai_uni = len(df_ai_uni)

# Transformations after union

## Dropping values based on project scope

<div class="c">
Uknown salary values were dropped already. <br>
But furthermore, we need to know the <b>Country, Seniority, Job title</b>. If any of those is missing, I'll drop the row from further investigation.
</div> 

### Employment status

#### AI-Jobs.net

In [ ]:
df_ai_uni['employment_status'].unique()

In [ ]:
len_ai_employmentdrop1 = len(df_ai_uni)
df_ai_uni = df_ai_uni[df_ai_uni['employment_status'] == 'ft']
len_ai_employmentdrop2 = len(df_ai_uni)

There are no students in this dataset, therefore I set the dropped student counter to 0.

In [ ]:
len_ai_studentdrop1 = 0
len_ai_studentdrop2 = 0

#### DE IT-Survey

<div class="c">
Dropping Nulls
</div> 

In [ ]:
df_it_uni['employment_status'][df_it_uni['year'] == 2020].unique()

In [ ]:
len_it_employmentdrop1 = len(df_it_uni)

# Define the years to filter
years_to_filter = [2020, 2021, 2022, 2023]

# Separate the rows where the year is in the specified range
df_filtered_years = df_it_uni[df_it_uni['year'].isin(years_to_filter)]

# Drop rows with NaN employment_status in the filtered subset
df_filtered_years = df_filtered_years.dropna(subset=['employment_status'])

# Combine back the filtered rows with the rest of the DataFrame
df_it_uni = pd.concat([df_filtered_years, df_it_uni[~df_it_uni['year'].isin(years_to_filter)]], ignore_index=True)

len_it_employmentdrop2 = len(df_it_uni)

In [ ]:
df_it_uni['employment_status'][df_it_uni['year'] == 2020].unique()

<div class="c">
Dropping freelance, parttime, student
</div> 

In [ ]:
df_it_uni['employment_status'].unique()

In [ ]:
df_it_uni = df_it_uni.reset_index(drop=True)

In [ ]:
len_it_studentdrop1 = len(df_it_uni)
df_it_uni = df_it_uni[df_it_uni['employment_status'].isin(['full-time employee', 'founder', 'full-time position, part-time position, & self-employed (freelancing, tutoring)', 'full/part-time employee']) | df_it_uni['employment_status'].isna()]
len_it_studentdrop2 = len(df_it_uni)

In [ ]:
df_it_uni['employment_status'].unique()

#### Kaggle

In [ ]:
df_k_uni['are_you_student'].unique()

In [ ]:
#df_k_uni[ (df_k_uni['are_you_student'].notna()) & (df_k_uni['are_you_student'] != 'no') ].head()

In [ ]:
# This drops row number 70k --> 10k !
# df_k_uni = df_k_uni[(df_k_uni['are_you_student'] == 'no')]

# By manual inspection it seems that many people left it unanswered. It's better to just filter out the explicit 'yes'.

In [ ]:
df_k_uni[df_k_uni['are_you_student'] == 'yes'].head()

<div class="c">
The dedicated 'are_you_student' column is **not** filled properly, therefore I omit this counter.<br>
Furthermore, there is no dedicated employment status category, therefore I also omit this counter.
</div>

In [ ]:
#len_k_studentdrop1 = 
#df_k_uni = df_k_uni[df_k_uni['are_you_student'].isin(['yes'])]
#len_k_studentdrop2 = 

<div class="c">
Dropping from 'experience' column
</div> 

In [ ]:
df_k_uni['experience'].unique()

In [ ]:
len_k_noncoderdrop1 = len(df_k_uni)
df_k_uni = df_k_uni[df_k_uni['experience'] != 'i have never written code']
len_k_noncoderdrop2 = len(df_k_uni)

### Country

#### AI-Jobs

In [ ]:
df_ai_uni['country'].sort_values().unique()

In [ ]:
len_ai_countrydrop1 = len(df_ai_uni)
df_ai_uni.dropna(subset=['country'], inplace=True)
len_ai_countrydrop2 = len(df_ai_uni)

#### Kaggle

In [ ]:
df_k_uni['country'].sort_values().unique()

In [ ]:
len_k_countrydrop1 = len(df_k_uni)
df_k_uni = df_k_uni[df_k_uni['country'] != 'i do not wish to disclose my location']
df_k_uni = df_k_uni[df_k_uni['country'] != 'other']
len_k_countrydrop2 = len(df_k_uni)

#### Germany IT-Survey

This is a germany-specific survey, therefore I just set the counter to 0.

In [ ]:
len_it_countrydrop1 = 0
len_it_countrydrop2 = 0

### Seniority_level

#### Ai-Jobs

In [ ]:
len_ai_senioritydrop1 = len(df_ai_uni)
df_ai_uni.dropna(subset=['seniority_level'], inplace=True)
len_ai_senioritydrop2 = len(df_ai_uni)

#### Germany IT-Survey

In [ ]:
len_it_senioritydrop1 = len(df_it_uni)
df_it_uni.dropna(subset=['seniority_level'], inplace=True)
len_it_senioritydrop2 = len(df_it_uni)

#### Kaggle

I'll later transform 'experience' into seniority, therefore, for the counter I add this to the senioritydrop

In [ ]:
len_k_senioritydrop1 = len(df_k_uni)
df_k_uni.dropna(subset=['experience'], inplace=True)
len_k_senioritydrop2 = len(df_k_uni)

### Job-title

#### AI-jobs.net

In [ ]:
len_ai_jobtitledrop1 = len(df_ai_uni)
df_ai_uni.dropna(subset=['job_title'], inplace=True)
len_ai_jobtitledrop2 = len(df_ai_uni)

#### Germany IT-Survey

In [ ]:
len_it_jobtitledrop1 = len(df_it_uni)
df_it_uni.dropna(subset=['job_title'], inplace=True)
len_it_jobtitledrop2 = len(df_it_uni)

#### Kaggle

In [ ]:
len_k_jobtitledrop1 = len(df_k_uni)
df_k_uni.dropna(subset=['job_title'], inplace=True)
len_k_jobtitledrop2 = len(df_k_uni)

## Uniformization

### Country Codes

#### Kaggle

In [ ]:
df_k_uni['country'].sort_values().unique()

In [ ]:
# Dictionary to map country names to 2-letter country codes
country_to_code = {
    'france': 'fr',
    'india': 'in',
    'indonesia': 'id',
    'united states of america': 'us',
    'australia': 'au',
    'mexico': 'mx',
    'germany': 'de',
    'turkey': 'tr',
    'netherlands': 'nl',
    'nigeria': 'ng',
    'canada': 'ca',
    'greece': 'gr',
    'belgium': 'be',
    'singapore': 'sg',
    'italy': 'it',
    'ireland': 'ie',
    'taiwan': 'tw',
    'russia': 'ru',
    'brazil': 'br',
    'south africa': 'za',
    'poland': 'pl',
    'iran, islamic republic of...': 'ir',
    'ukraine': 'ua',
    'pakistan': 'pk',
    'chile': 'cl',
    'japan': 'jp',
    'egypt': 'eg',
    'south korea': 'kr',
    'belarus': 'by',
    'viet nam': 'vn',
    'colombia': 'co',
    'israel': 'il',
    'china': 'cn',
    'united kingdom of great britain and northern ireland': 'gb',
    'sweden': 'se',
    'bangladesh': 'bd',
    'portugal': 'pt',
    'tunisia': 'tn',
    'argentina': 'ar',
    'czech republic': 'cz',
    'spain': 'es',
    'hong kong (s.a.r.)': 'hk',
    'cameroon': 'cm',
    'saudi arabia': 'sa',
    'austria': 'at',
    'kenya': 'ke',
    'morocco': 'ma',
    'romania': 'ro',
    'hungary': 'hu',
    'republic of korea': 'kr',
    'norway': 'no',
    'ethiopia': 'et',
    'philippines': 'ph',
    'thailand': 'th',
    'denmark': 'dk',
    'switzerland': 'ch',
    'peru': 'pe',
    'sri lanka': 'lk',
    'ghana': 'gh',
    'malaysia': 'my',
    'united arab emirates': 'ae',
    'nepal': 'np',
    'iraq': 'iq',
    'new zealand': 'nz',
    'algeria': 'dz',
    'ecuador': 'ec',
    'uganda': 'ug',
    'kazakhstan': 'kz',
    'zimbabwe': 'zw',
    'latvia': 'lv'
}

In [ ]:
# Transform country names to 2-letter country codes
df_k_uni['country'] = df_k_uni['country'].map(country_to_code)
df_k_uni['country'].unique()

### Seniority level

#### De-IT

In [ ]:
df_it_uni['seniority_level'].unique()

In [ ]:
# Define the mapping dictionary
seniority_mapping_it = {
    'head / principal': 'executive',
    'lead / staff': 'executive',
    'c-level executive manager': 'executive',
    'head': 'executive',
    'lead': 'executive',
    'director': 'executive',
    'manager': 'executive',
    'vp': 'executive',
    'c-level executive manager':'executive',
    'cto': 'executive',  # direCTOr
    'principal': 'executive',
    'c-level': 'executive',
    'middle': 'medior',
    'entry level': 'junior',
    'intern': 'other',
    'working student': 'other',
    'student': 'other',
    '800': 'other',
    'key': 'other',
    'no idea, there are no ranges in the firm ': 'other',
    'self employed': 'other',
    'work center executive': 'other',
    'no level ': 'other',
    'no level': 'other',
    'work center manager':'other'
}

In [ ]:
# Replace the seniority levels using the mapping dictionary
df_it_uni['seniority_level'] = df_it_uni['seniority_level'].replace(seniority_mapping_it)
df_it_uni['seniority_level'].unique()

#### AI-Jobs.net

In [ ]:
df_ai_uni['seniority_level'].unique()

In [ ]:
# Define the mapping dictionary
seniority_mapping_ai = {
    'mi':'medior',
    'en': 'junior',
    'se': 'senior',
    'ex': 'executive'
}

In [ ]:
# Replace the seniority levels using the mapping dictionary
df_ai_uni['seniority_level'] = df_ai_uni['seniority_level'].replace(seniority_mapping_ai)
#df_ai_uni['seniority_level'] = df_ai_uni['seniority_level'].cat.rename_categories(seniority_mapping_ai)
df_ai_uni['seniority_level'].unique()

#### Kaggle

In [ ]:
df_k_uni['experience'].unique()

In [ ]:
# Mapping of experience intervals to seniority levels
experience_to_seniority = {
    '< 1 years': 'junior',
    '1-2 years': 'junior',
    '1-3 years': 'junior',
    '3-5 years': 'medior',
    '5-10 years': 'senior',
    '10-20 years': 'senior',
    '20+ years': 'executive'
}

In [ ]:
# Create the seniority_level column
df_k_uni['seniority_level'] = df_k_uni['experience'].map(experience_to_seniority)
df_k_uni.head()

## Additonal cleaning

### Germany-IT

In [ ]:
df_it_uni.head(1)

#### 'experience' and 'years_of_experience_in_germany'

<div class="c">
'experience' and 'years_of_experience_in_germany' columns are filled with unclean answers
</div> 

In [ ]:
df_it_uni['experience'].unique()

In [ ]:
df_it_uni['years_of_experience_in_germany'].unique()

In [ ]:
df_report_additional = pd.DataFrame(index=[0])

In [ ]:
df_it_uni['experience'] = df_it_uni['experience'].apply(clean_salary_0616, df_report=df_report_additional, prefix='experience')
df_report_additional

In [ ]:
df_it_uni['experience'] = df_it_uni['experience'].apply(pd.to_numeric, errors='coerce')
df_it_uni['experience'] = df_it_uni['experience'][(df_it_uni['experience'] <= 100)]

In [ ]:
df_it_uni['years_of_experience_in_germany'] = df_it_uni['years_of_experience_in_germany'].apply(clean_salary_0616, df_report=df_report_additional, prefix='experience_in_de')
df_report_additional

In [ ]:
df_it_uni['years_of_experience_in_germany'] = df_it_uni['years_of_experience_in_germany'].apply(pd.to_numeric, errors='coerce')
df_it_uni['years_of_experience_in_germany'] = df_it_uni['years_of_experience_in_germany'][(df_it_uni['years_of_experience_in_germany'] <= 100)]

In [ ]:
df_it_uni['experience'].describe()

#### City

In [ ]:
df_it_uni.groupby('city')['salary'].count().sort_values(ascending=False).head(6)

In [ ]:
df_it_uni.loc[df_it_uni['city'] == 'münchen', 'city'] = 'munich'

In [ ]:
df_it_uni.groupby('city')['salary'].count().sort_values(ascending=False).head(6)

In [ ]:
# Define the major cities to keep
major_cities = ['berlin', 'munich', 'frankfurt', 'hamburg', 'stuttgart']

# Create the 'city_category' column
df_it_uni['city_category'] = df_it_uni['city'].apply(lambda x: x if x in major_cities else 'other')

#### Language at work

In [ ]:
df_it_uni['language_at_work'].unique()

In [ ]:
def categorize_language(language_entry, categories):
    language_entry = str(language_entry).lower()  # Convert language_entry to lower case string
    for category, keywords in categories.items():
        for keyword in keywords:
            pattern = re.escape(keyword.lower())  # Create regex pattern for keyword
            if re.search(pattern, language_entry):
                return category
    return 'Only other languages'  # For entries that don't match any category

In [ ]:
language_categories = {
    'German-speaking': ['german', 'deutsch'],
    'English-speaking (but not german)': ['english']
}

df_it_uni['language_category'] = df_it_uni['language_at_work'].apply(lambda x: categorize_language(x, language_categories))

In [ ]:
df_it_uni['language_category'].unique()

In [ ]:
df_it_uni.groupby('language_category')['salary'].count().sort_values(ascending=False).head(6)

#### company_size

In [ ]:
df_it_uni.groupby('company_size')['salary'].count().sort_values(ascending=False)

In [ ]:
df_it_uni['company_size'] = df_it_uni['company_size'].replace({'10-50': '11-50'})
df_it_uni['company_size'] = df_it_uni['company_size'].replace({'50-100': '51-100'})
df_it_uni['company_size'] = df_it_uni['company_size'].replace({'100-1000': '101-1000'})

In [ ]:
df_it_uni.groupby('company_size')['salary'].count().sort_values(ascending=False)

In [ ]:
def categorize_company_size(size):
    """
    Categorize company size into small (s), medium (m), or large (l).
    Handles NaN values and unknown categories.
    
    Parameters:
    size: Company size value (string or NaN)
    
    Returns:
    string: 's', 'm', 'l', or 'unknown'
    """
    # Handle NaN values first using pandas.isna()
    if pd.isna(size):
        return 'unknown'
        
    # Convert to string to handle any numeric inputs
    size = str(size).lower().strip()
    
    # Direct mapping for known categories
    if size in ['s', 'm', 'l']:
        return size
        
    # Large companies
    elif size in ['1000+', '1000-9,999 employees', '10,000 or more employees', 
                 '> 10,000 employees', '>1000', '1000 or more employees']:
        return 'l'
        
    # Medium companies
    elif size in ['101-1000', '250-999 employees', '50-249 employees', '51-100',
                 '100-999', '51-1000']:
        return 'm'
        
    # Small companies
    elif size in ['11-50', 'up to 10', '0-49 employees', '<50', '1-50',
                 '0-50', 'under 50 employees']:
        return 's'
        
    # Any other value is considered unknown
    return 'unknown'

In [ ]:
# Apply the function to both DataFrames
df_it_uni['company_size_category'] = df_it_uni['company_size'].apply(categorize_company_size)
df_it_uni['company_size_category'].unique()

#### 'company_industry'

In [ ]:
df_it_uni.groupby('company_industry')['salary'].count().sort_values(ascending=False)

In [ ]:
# Define the function to categorize industries
def categorize_industry(industry):
    if industry in ['information services, it, software development, or other technology']:
        return 'information technology'
    elif industry in ['financial services', 'insurance']:
        return 'financial services'
    elif industry == 'retail and consumer services':
        return 'retail and consumer services'
    elif industry == 'manufacturing, transportation, or supply chain':
        return 'manufacturing, transportation, or supply chain'
    elif industry == 'healthcare':
        return 'healthcare'
    else:
        return 'other'

# Create the 'industry_category' column
df_it_uni['industry_category'] = df_it_uni['company_industry'].apply(categorize_industry)

#### company_type

In [ ]:
len(df_it_uni['company_type'].unique())

<div class="c">
This is a free-string cell. I will not try to make sense of it; <br>
Feel free to look inside... the required effort for this is out of scope now.
</div> 

### Kaggle

In [ ]:
df_k_uni.head(1)

#### Education level

In [ ]:
df_k_uni['education_level'].unique()

In [ ]:
df_k_uni['education_level'] = df_k_uni['education_level'].str.replace('no formal education past high school','no degree')
df_k_uni['education_level'] = df_k_uni['education_level'].str.replace('some college/university study without earning a bachelor’s degree','no degree')
df_k_uni = df_k_uni[(df_k_uni['education_level'] == 'bachelor’s degree') | (df_k_uni['education_level'] == 'master’s degree') | (df_k_uni['education_level'] == 'doctoral degree') | (df_k_uni['education_level'] == 'no degree')]

#### company_size

In [ ]:
df_k_uni.groupby('company_size')['salary'].count().sort_values(ascending=False)

In [ ]:
df_k_uni['company_size_category'] = df_k_uni['company_size'].apply(categorize_company_size)
df_k_uni['company_size_category'].unique()

#### experience

In [ ]:
df_k_uni.groupby('experience')['salary'].count().sort_values(ascending=False)

In [ ]:
df_k_uni['experience'] = df_k_uni['experience'].replace({'1-2 years': '1-3 years'})

#### industry

In [ ]:
df_k_uni.groupby('industry')['salary'].count().sort_values(ascending=False)

## AI-jobs

### Company size

In [ ]:
# Remove rows where all columns are NaN
df_ai_uni = df_ai_uni.dropna(how='all')

In [ ]:
df_ai_uni['company_size_category'] = df_ai_uni['company_size'].apply(categorize_company_size)
df_ai_uni['company_size_category'].unique()

In [ ]:
df_ai_uni[df_ai_uni['company_size'].isna()].head()

# Deriving new variables

<div class="c">
I create new categories to simplify the analysis later.<br>
For example, the country-standardization by GDP-per-capita might flatten out the huge variance when it comes to country dependencies. <br>
This is all in hope that I might be able to drop the country parameter.<br>
The idea for this chapter, chronologically was born deep in the analysis/ prediction phase. But was inserted here for the sake of project flow.

</div> 

## Country-standardized salary

In [ ]:
# GDP per capita data (in USD)
gdp_per_capita = {
    'fr': 40886, 'in': 2016, 'au': 65099, 'us': 76329, 'nl': 57025, 
    'de': 48717, 'ie': 87947, 'ru': 15270, 'gr': 19829, 'ua': 4533, 
    'pk': 1491, 'jp': 40066, 'br': 8697, 'kr': 31961, 
    'by': 7888, 'ng': 2229, 'gb': 46125, 'se': 53755, 'mx': 10657, 
    'ca': 54917, 'pt': 23758, 'pl': 17939, 'id': 4289, 'it': 34776, 
    'cz': 23906, 'es': 31688, 'cl': 14938, 'hk': 46544, 'za': 6001, 
    'ar': 10461, 'tr': 10674, 'il': 44162, 'tw': 34166, 'eg': 3801, 
    'ma': 3585, 'hu': 18390, 'co': 6214, 'no': 89111, 'th': 7775, 
    'ch': 93259, 'vn': 3704, 'sg': 59806, 'bd': 1964, 'ir': 2273, 
    'pe': 7002, 'ke': 2066, 'ro': 15786, 'cn': 12710, 'be': 50126, 
    'at': 52084, 'dz': 4094, 'nz': 45380, 'tn': 3840, 'ph': 3593, 
    'my': 11109, 'dk': 61612, 'sa': 20619, 'ae': 43103, 'np': 1192, 
    'lk': 3841, 'gh': 2396, 'et': 936, 'iq': 4922, 'ec': 6245, 
    'kz': 10153, 'ug': 817, 'cm': 1500, 'zw': 1098, 
    'lv': 21779, 'ge': 4804, 'lt': 25064, 'fi': 53012, 'hr': 15647, 'om': 25056, 'ba': 7568, 'ee': 28247, 
    'mt': 34127, 'lb': 4136, 'si': 28439, 'mu': 10256, 'am': 7018, 'qa': 87661, 'ad': 41992, 'md': 5714,
    'uz': 2255, 'cf': 427, 'kw': 41079, 'cy': 32048, 'as': 19673, 'cr': 13365, 'pr': 35208, 'bo': 3600,
    'do': 8793, 'hn': 2736, 'bg': 12623, 'je': 55820, 'rs': 9260,  'lu': 125006
}

In [ ]:
country_salary_stats

In [ ]:
#df_ai_uni.drop(['country_code', 'median_income_2020_usd', 'mean_income_2020_usd', 'gdp_ppp_usd', 'glassdoor_software_engineer_usd','salary_normmed','salary_normmean','salary_normgdp','salary_normse'], axis=1, inplace=True)

In [ ]:
#df_k_uni.drop(['country_code', 'median_income_2020_usd', 'mean_income_2020_usd', 'gdp_ppp_usd', 'glassdoor_software_engineer_usd','salary_normmed','salary_normmean','salary_normgdp','salary_normse'], axis=1, inplace=True)

In [ ]:
#df_it_uni.drop(['country_code', 'median_income_2020_usd', 'mean_income_2020_usd', 'gdp_ppp_usd', 'glassdoor_software_engineer_usd','salary_normmed','salary_normmean','salary_normgdp','salary_normse'], axis=1, inplace=True)

In [ ]:
df_name = df_ai_uni

# Merge df with country_salary_stats to get the median income for each country
df_name = df_name.merge(
    country_salary_stats[['country_code', 'median_income_2020_usd', 'mean_income_2020_usd', 'gdp_ppp_usd', 'glassdoor_software_engineer_usd']],
    left_on='country',
    right_on='country_code',
    how='left'
)

# Calculate the normalized salaries
df_name['salary_normmed'] = df_name['salary'] / df_name['median_income_2020_usd']
df_name['salary_normmean'] = df_name['salary'] / df_name['mean_income_2020_usd']
df_name['salary_normgdp'] = df_name['salary'] / df_name['gdp_ppp_usd']
df_name['salary_normse'] = df_name['salary'] / df_name['glassdoor_software_engineer_usd']

df_ai_uni = df_name

In [ ]:
df_name = df_k_uni

# Merge df with country_salary_stats to get the median income for each country
df_name = df_name.merge(
    country_salary_stats[['country_code', 'median_income_2020_usd', 'mean_income_2020_usd', 'gdp_ppp_usd', 'glassdoor_software_engineer_usd']],
    left_on='country',
    right_on='country_code',
    how='left'
)

# Calculate the normalized salaries
df_name['salary_normmed'] = df_name['salary'] / df_name['median_income_2020_usd']
df_name['salary_normmean'] = df_name['salary'] / df_name['mean_income_2020_usd']
df_name['salary_normgdp'] = df_name['salary'] / df_name['gdp_ppp_usd']
df_name['salary_normse'] = df_name['salary'] / df_name['glassdoor_software_engineer_usd']

df_k_uni = df_name

In [ ]:
df_name = df_it_uni

# Merge df with country_salary_stats to get the median income for each country
df_name = df_name.merge(
    country_salary_stats[['country_code', 'median_income_2020_usd', 'mean_income_2020_usd', 'gdp_ppp_usd', 'glassdoor_software_engineer_usd']],
    left_on='country',
    right_on='country_code',
    how='left'
)

# Calculate the normalized salaries
df_name['salary_normmed'] = df_name['salary'] / df_name['median_income_2020_usd']
df_name['salary_normmean'] = df_name['salary'] / df_name['mean_income_2020_usd']
df_name['salary_normgdp'] = df_name['salary'] / df_name['gdp_ppp_usd']
df_name['salary_normse'] = df_name['salary'] / df_name['glassdoor_software_engineer_usd']

df_it_uni = df_name

### Approach 2

In [ ]:
df_ai_uni['country'][~(df_ai_uni['country'].isin(gdp_per_capita))].unique()

In [ ]:
# Normalize the salary - Kaggle
df_k_uni['salary_norm'] = df_k_uni.apply(lambda x: x['salary'] / gdp_per_capita[x['country']], axis=1)
df_k_uni.head(2)

In [ ]:
# Normalize the salary - AI-Jobs.net
df_ai_uni['salary_norm'] = df_ai_uni.apply(lambda x: x['salary'] / gdp_per_capita[x['country']], axis=1)
df_ai_uni.head()

In [ ]:
# Normalize the salary - Germany IT-Survey
df_it_uni['salary_norm'] = df_it_uni.apply(lambda x: x['salary'] / gdp_per_capita[x['country']], axis=1)
df_it_uni.head()

## Year-Standardized salary

In [ ]:
inflation_rates = {
    2018: 1.019,  # USD inflation from 2018 to 2019 (22.66% increase from 2018 to 2024)
    2019: 1.018,  # USD inflation from 2019 to 2020 (20.37% increase from 2019 to 2024)
    2020: 1.012,  # USD inflation from 2020 to 2021 (18.25% increase from 2020 to 2024)
    2021: 1.040,  # USD inflation from 2021 to 2022 (16.84% increase from 2021 to 2024)
    2022: 1.070,  # USD inflation from 2022 to 2023 (12.35% increase from 2022 to 2024)
    2023: 1.050,  # USD inflation from 2023 to 2024 (5.00% increase from 2023 to 2024)
    2024: 1.000   # base year, no inflation adjustment for 2024, 
}

# Calculate cumulative inflation adjustment factor
def calculate_cumulative_inflation(start_year, end_year=2024):
    if start_year >= end_year:
        return 1.0
    inflation_factors = [inflation_rates[year] for year in range(start_year, end_year)]
    cumulative_inflation = 1.0
    for factor in inflation_factors:
        cumulative_inflation *= factor
    return cumulative_inflation

# Apply inflation adjustment to the salary column
def salary_to_2024(row):
    return row['salary'] * calculate_cumulative_inflation(row['year'])

# Apply inflation adjustment to the salary column
def salarynorm_to_2024(row):
    return row['salary_norm'] * calculate_cumulative_inflation(row['year'])

In [ ]:
# Apply inflation adjustment directly to the 'salary' and 'salary_norm' columns
df_it_uni['salary_2024'] = df_it_uni.apply(lambda row: row['salary'] * calculate_cumulative_inflation(row['year']), axis=1)
df_it_uni['salary_norm_2024'] = df_it_uni.apply(lambda row: row['salary_norm'] * calculate_cumulative_inflation(row['year']), axis=1)
df_it_uni['salary_normmed_2024'] = df_it_uni.apply(lambda row: row['salary_normmed'] * calculate_cumulative_inflation(row['year']), axis=1)
df_it_uni['salary_normmean_2024'] = df_it_uni.apply(lambda row: row['salary_normmean'] * calculate_cumulative_inflation(row['year']), axis=1)
df_it_uni['salary_normgdp_2024'] = df_it_uni.apply(lambda row: row['salary_normgdp'] * calculate_cumulative_inflation(row['year']), axis=1)
df_it_uni['salary_normse_2024'] = df_it_uni.apply(lambda row: row['salary_normse'] * calculate_cumulative_inflation(row['year']), axis=1)

#df_it_uni['salary_2024'] = df_it_uni.apply(salary_to_2024, axis=1)
#df_it_uni['salary_norm_2024'] = df_it_uni.apply(salarynorm_to_2024, axis=1)
#df_it_uni['salary_normmed_2024'] = df_it_uni.apply(salarynorm_to_2024, axis=1)
#df_it_uni['salary_normmean_2024'] = df_it_uni.apply(salarynorm_to_2024, axis=1)
#df_it_uni['salary_normgdp_2024'] = df_it_uni.apply(salarynorm_to_2024, axis=1)
#df_it_uni['salary_normse_2024'] = df_it_uni.apply(salarynorm_to_2024, axis=1)

# Display the updated dataframe
df_it_uni[['year', 'salary', 'salary_2024', 'salary_norm', 'salary_norm_2024', 'salary_normmed_2024', 'salary_normmean_2024', 'salary_normgdp_2024', 'salary_normse_2024']].head()

In [ ]:
# Apply inflation adjustment directly to the 'salary' and 'salary_norm' columns
df_k_uni['salary_2024'] = df_k_uni.apply(lambda row: row['salary'] * calculate_cumulative_inflation(row['year']), axis=1)
df_k_uni['salary_norm_2024'] = df_k_uni.apply(lambda row: row['salary_norm'] * calculate_cumulative_inflation(row['year']), axis=1)
df_k_uni['salary_normmed_2024'] = df_k_uni.apply(lambda row: row['salary_normmed'] * calculate_cumulative_inflation(row['year']), axis=1)
df_k_uni['salary_normmean_2024'] = df_k_uni.apply(lambda row: row['salary_normmean'] * calculate_cumulative_inflation(row['year']), axis=1)
df_k_uni['salary_normgdp_2024'] = df_k_uni.apply(lambda row: row['salary_normgdp'] * calculate_cumulative_inflation(row['year']), axis=1)
df_k_uni['salary_normse_2024'] = df_k_uni.apply(lambda row: row['salary_normse'] * calculate_cumulative_inflation(row['year']), axis=1)

# Display the updated dataframe
df_k_uni[['year', 'salary', 'salary_2024', 'salary_norm', 'salary_norm_2024', 'salary_normmed_2024', 'salary_normmean_2024', 'salary_normgdp_2024', 'salary_normse_2024']].head()

In [ ]:
df_ai_uni.dropna(subset=['year'], inplace=True)

In [ ]:
df_ai_uni['year'] = df_ai_uni['year'].astype('int64')

In [ ]:
# Apply inflation adjustment directly to the 'salary' and 'salary_norm' columns
df_ai_uni['salary_2024'] = df_ai_uni.apply(lambda row: row['salary'] * calculate_cumulative_inflation(row['year']), axis=1)
df_ai_uni['salary_norm_2024'] = df_ai_uni.apply(lambda row: row['salary_norm'] * calculate_cumulative_inflation(row['year']), axis=1)
df_ai_uni['salary_normmed_2024'] = df_ai_uni.apply(lambda row: row['salary_normmed'] * calculate_cumulative_inflation(row['year']), axis=1)
df_ai_uni['salary_normmean_2024'] = df_ai_uni.apply(lambda row: row['salary_normmean'] * calculate_cumulative_inflation(row['year']), axis=1)
df_ai_uni['salary_normgdp_2024'] = df_ai_uni.apply(lambda row: row['salary_normgdp'] * calculate_cumulative_inflation(row['year']), axis=1)
df_ai_uni['salary_normse_2024'] = df_ai_uni.apply(lambda row: row['salary_normse'] * calculate_cumulative_inflation(row['year']), axis=1)

# Display the updated dataframe
df_ai_uni[['year', 'salary', 'salary_2024', 'salary_norm', 'salary_norm_2024', 'salary_normmed_2024', 'salary_normmean_2024', 'salary_normgdp_2024', 'salary_normse_2024']].head()

## Categorizing Job-titles

In [ ]:
#df_it['job_category_kw'] = df_it['job_title'].apply(categorize_by_keywords, category_keywords)
## df_it['job_category_kw'] = df_it['job_title'].apply(lambda x: categorize_by_keywords_1627(x, category_keywords))
#df_it['job_category'] = df_it['job_title'].apply(lambda x: categorize_job_title(x, categories))

In [ ]:
# Function to categorize job titles based on keyword match
def categorize_job_title_1945(job_title, categories):
    job_title = str(job_title).lower()  # Convert job_title to lower case string
    for category, keywords in categories.items():
        for keyword in keywords:
            pattern = re.escape(keyword.lower())  # Create regex pattern for keyword
            if re.search(pattern, job_title):
                return category
    return 'Uncategorized'  # For job titles that don't match any category

In [ ]:
job_categories = {
    'Project managers': [
        'project manager', 'pm' , 'program manager', 'project manager ', 'projectingenieur', 'project manager (pm)', 'program/project manager', 'technical lead', 
        'project manager & scrum master', 'technical project manager', 'digital project manager', 'it project manager', 
        'scrum master', 'scrum master / agile coach', 'sr project manager', 'senior project manager', 'senior program manager', 'engineering project manager', 
        'agile project manager', 'project leader', 'director of engineering', 'director of project management', 'director of technology', 'director of operations', 
        'project consultant', 'project coordinator', 'project supervisor', 'project assistant', 'project administrator', 'project management officer', 
        'program manager (technical)', 'construction project manager', 'it program manager', 'it project coordinator', 'it project management consultant', 
        'it project manager ', 'associate project manager', 'project portfolio manager', 'project office manager', 'product manager', 
        'technical program manager', 'digital transformation project manager', 'technical program manager (tpm)', 'digital project lead', 'delivery manager', 
        'global project manager', 'global program manager', 'business program manager', 'service delivery manager', 'it delivery manager', 'operations project manager', 
        'customer project manager', 'implementation project manager', 'senior delivery manager', 'business development manager operations', 'project & operations manager', 
        'it operations manager', 'manager (program, project, operations, executive-level, etc)', 'project & operations manager', 'technical project lead'
    ],
    'Team leaders': ['team lead', 'team leader'],
    'Leaders': ['head of', 'lead', 'principal', 'staff', 'vp', 'cto'],
    'Other managers': ['manager'],
    'Full Stack Developers': ['full stack', 'full-stack', 'fullstack'],
    'Architects': ['architect', 'data modeler', 'architekt'],
    'Cloud': ['cloud engineer', 'cloud consulting', 'cloud platform engineer', 'cloud infrastructure engineer', 'cloud automation engineer'],
    
    'PHP Developers': ['php'],
    'SAP Specialists': ['sap'],
    'NET Developers': ['.net', 'c#'],
    'C++ Developers': ['c++'],
    'Mobile': ['ios','mobile', 'android','application'],
    'Java/Scala Developers': ['java', 'scala','javascript','js', 'angular'],
    'Other languages': ['python', 'ruby', 'oracle', 'erlang', 'go', 'golang', 'pyhon'],
    
    'Embedded Engineers': ['embedded'],
    'Front End': ['front end', 'front-end','frontend','frontent'],
    'Back End': ['back end','back-end', 'backend'],
    'Web developer': ['web developer', 'webdev', 'web-entwickler'],
    'Game': ['unreal','game','unity','unity3d'],
    'Hardware':['hardware'],
    'Security':['security'],
    'Database Dev & Admin':['dba', 'database developer', 'database engineer', 'database administrator', 'database manager', 'databengineer', 'data administrator'],
    'System admin': ['sys admin', 'sysadmin', 'system administrator', 'systems administrator', 'it administration', 'it admin', 'network administrator'],
    'Statisticians': ['statistician'],
    'Consultant': ['consultant', 'berater', 'consulter', 'consulting'],
    
    'Researcher': ['researcher'],
    'Prompt Engineer': ['prompt'],
    'Bioinformatics': ['bioinformatics', 'biostatistics', 'computational biologist'],
    
    'Business Analyst': [
        'business analyst', 'business intelligence analyst', 'bi analyst', 'bi specialist',
        'business insights analyst', 'financial data analyst', 'compliance data analyst', 'product data analyst', 'marketing data analyst', 'business data analyst', 
        'data analyst (business, marketing, financial, quantitative, etc)', 'business intelligence manager', 'business intelligence engineer', 'business intelligence specialist', 
        'business intelligence data analyst', 'business analyst/re', 'business analyst ', 'business analyst / business development manager operations', 
        'business development manager operations'
    ],
    
    'Data Analyst': [
        'data analyst', 'business intelligence developer', 'bi developer', 'research analyst', 'analytics engineer', 'data management analyst', 'data visualization',
        'data strategist',
        'data analytics associate', 'product analyst', 'marketing analyst', 'dana analyst', 'data reporting analyst', 'data quality analyst', 'finance data analyst', 
        'compliance data analyst', 'product data analyst', 'marketing data analyst', 'financial data analyst', 'data analytics consultant', 'data integration analyst', 
        'insight analyst', 'data analyst (business, marketing, financial, quantitative, etc)', 'business data analyst', 'data modeller', 'data analytics manager', 
        'data operations analyst', 'data quality manager', 'data science analyst', 'data specialist', 'data strategy manager', 'data management consultant', 
        'data analytics lead', 'data analytics specialist', 'data operations manager', 'data product analyst', 'data product owner', 'data quality engineer',
        'data visualization analyst', 'data visualization specialist'
    ],
    
    'Business Analyst': [
        'business intelligence',
    ],
    
    'Data Engineer': [
        'ml ops engineer', 'data engineer', 'database engineer', 'big data engineer', 'etl developer', 'etl engineer', 'big data developer', 'data operations',
        'machine learning operations engineer','machine learning infrastructure engineer', 'machine learning developer',
        'data integration', 'data processing', 'data developer', 'data integration engineer', 'data pipeline engineer', 'cloud data engineer', 'data infrastructure engineer', 
        'data warehouse engineer', 'data migration engineer', 'etl/data engineer', 'data platform engineer', 'data ops engineer', 'data services engineer', 
        'data solutions engineer', 'data systems engineer', 'data automation engineer', 'data engineering manager', 'data engineer lead', 'data engineering consultant', 
        'data operations engineer', 'data modeling engineer', 'data engineering analyst', 'data warehouse developer', 'data engineer/scientist', 'data quality engineer', 
        'data mining engineer', 'data software engineer', 'data engineering specialist'
    ],
    
    'Data Scientist/ ML Engineer': [
        'ai developer', 'deep learning engineer', 'data science', 'data scientist', 'machine learning engineer', 'ai engineer', 'ml engineer', 'research scientist', 'deep learning engineer', 
        'machine learning specialist', 'ai programmer', 'ai scientist', 'decision scientist',
        'nlp engineer', 'computer vision engineer', 'applied scientist', 'ai/ml engineer', 'data scientist/analyst', 'data science engineer', 'data science analyst', 
        'data science manager', 'machine learning scientist', 'applied ml engineer', 'ai/ml scientist', 'research engineer', 'mlops engineer', 'data scientist lead', 
        'data scientist manager', 'senior data scientist', 'principal data scientist', 'staff machine learning engineer', 'staff data scientist', 'machine learning software engineer',
        'machine learning manager', 'machine learning ops engineer', 'principal machine learning engineer', 'applied machine learning engineer', 'ml engineer/analyst', 
        'data & applied scientist', 'machine learning research engineer', 'machine learning modeler', 'ai/ml researcher', 'mlops/data scientist', 'ai/ ml research engineer',
        'ml engineer/research scientist', 'ai/ml engineer/researcher', 'machine learning engineet'
    ],
    
    'Data Governance & Compliance': [
        'data governance specialist', 'data governance manager', 'compliance data analyst', 'data quality manager', 'data quality analyst', 'data quality engineer', 'data management analyst', 'data management consultant', 'data management specialist', 'data privacy officer', 'data protection officer', 'data security officer', 'data compliance manager', 'risk and compliance analyst', 'risk and compliance manager', 'compliance officer', 'data stewardship specialist', 'data steward', 'regulatory compliance analyst', 'regulatory compliance manager', 'information governance specialist', 'information governance manager', 'data governance consultant'
    ],
    
    'Software Engineer':[
        'software engineer', 'software developer'
    ],
    
    'DevOps Engineer': [
        'devops engineer', 'devops', 'devops engineer ',
        'software engineer (devops)', 'system engineer', 'system administrator', 'systems engineer', 
        'it infrastructure consultant', 'it consultant', 'solution engineer', 'lead devops', 'lead devops engineer', 'technical lead devops', 
        'sr. devops', 'sr. engineer', 'sre', 'sre engineer', 'site reliability engineer', 'site reliability engineer '
    ],
    
    'UI/UX Designers': [
        'ux designer', 'ui designer', 'ux/ui designer', 'designer (ui, ux)', 'designer (ui/ux)', 'product designer', 'interaction designer', 'user experience designer', 'visual designer', 'frontend designer', 'creative designer', 'web designer', 'graphic designer', 'mobile designer', 'user interface designer', 'design lead', 'design researcher', 'ui/ux specialist', 'ux/ui lead', 'digital designer', 'ui/ux consultant'
    ],
    
    'QA/Test Engineers': [
        'qa', 'testing', 'tester', 'test', 'qa test engineer', 'qa engineer', 'qa automation engineer', 'automation qa', 'test automation engineer', 'manual qa', 
        'qa automation', 'qa consultant', 'qa analyst', 'software test engineer', 'quality engineer', 'test engineer', 'test manager', 'testautomation', 
        'automation test engineer', 'quality assurance', 'quality assurance engineer', 'qa specialist', 'qa automation specialist', 'qa automation'
    ],
    
    'Other Engineers': [
        'platform engineer', 'engineer (non-software)', 'network engineer', 'support engineer', 'electrical engineer', 'firmware engineer', 'robotics engineer', 'it engineer', 'reporting engineer', 'ta engineer', 'cisco engineer'     
    ],
    
    'Other Developers': [
        'web developer', 'sw developer', 'softwaredeveloper', 'xr', 'crypto', 'rpa developer', 'sharepoint developer', 'nav developer', 'dwh developer', 'web deleloper', 'erp developer'
    ],
    
    'Out of scope': [
        'teacher', 'professor', 'lawyer', 'sales', 'pcb designer', 'coach', 'producer', 'recruiter', 'agile', 'banker', 'quant'
    ],
    'System...': [
        'system'
    ],
    'Advocacy':['developer advocate', 'developer relations/advocacy', ],
    
    'Too vague answers': [
        'engineer', 'developer', 'designer', 'support', 'operations', 'analyst', 'spezialist', 'specialist'
    ],
    '"Other"': ['other']
    
}

### DE IT-Survey

In [ ]:
df_it_uni['job_title'] = df_it_uni['job_title'].str.replace('senior', '', case=False, regex=False)
df_it_uni['job_title'] = df_it_uni['job_title'].str.replace('sr.', '', case=False, regex=False)
df_it_uni['job_title'] = df_it_uni['job_title'].str.strip()

In [ ]:
df_it_uni['job_category'] = df_it_uni['job_title'].apply(lambda x: categorize_job_title_1945(x, job_categories))

In [ ]:
# Plot histogram for the 'job_category' column
plt.figure(figsize=(8, 3))
df_it_uni['job_category'].value_counts().plot(kind='bar')
plt.title('Histogram of Job Categories')
plt.xlabel('Job Category')
plt.ylabel('Frequency')
plt.xticks(rotation=90)
plt.show()

In [ ]:
df_it_uni['job_title'][df_it_uni['job_category'] == 'Uncategorized'].tail(50)
#df_it_uni['job_title'][df_it_uni['job_category'] == 'Too vague answers'].tail(50)

In [ ]:
df_it_uni['job_title'][df_it_uni['job_category'] == 'Database Development/ Admininistration'].head(50)

### AI-Jobs.net

In [ ]:
df_ai_uni['job_title'] = df_ai_uni['job_title'].str.replace('senior', '', case=False, regex=False)
df_ai_uni['job_title'] = df_ai_uni['job_title'].str.replace('sr.', '', case=False, regex=False)
df_ai_uni['job_title'] = df_ai_uni['job_title'].str.strip()

In [ ]:
df_ai_uni['job_category'] = df_ai_uni['job_title'].apply(lambda x: categorize_job_title_1945(x, job_categories))

In [ ]:
# Plot histogram for the 'job_category' column
plt.figure(figsize=(8, 3))
df_ai_uni['job_category'].value_counts().plot(kind='bar')
plt.title('Histogram of Job Categories')
plt.xlabel('Job Category')
plt.ylabel('Frequency')
plt.xticks(rotation=90)
plt.show()

In [ ]:
df_ai_uni['job_title'][df_ai_uni['job_category'] == 'Uncategorized'].head(50)
#df_ai_uni['job_title'][df_ai_uni['job_category'] == 'Too vague answers'].tail(50)

### Kaggle

In [ ]:
df_k_uni['job_title'] = df_k_uni['job_title'].str.replace('senior', '', case=False, regex=False)
df_k_uni['job_title'] = df_k_uni['job_title'].str.replace('sr.', '', case=False, regex=False)
df_k_uni['job_title'] = df_k_uni['job_title'].str.strip()

In [ ]:
df_k_uni['job_category'] = df_k_uni['job_title'].apply(lambda x: categorize_job_title_1945(x, job_categories))

In [ ]:
# Plot histogram for the 'job_category' column
plt.figure(figsize=(8, 3))
df_k_uni['job_category'].value_counts().plot(kind='bar')
plt.title('Histogram of Job Categories')
plt.xlabel('Job Category')
plt.ylabel('Frequency')
plt.xticks(rotation=90)
plt.show()

In [ ]:
df_k_uni['job_title'][df_k_uni['job_category'] == 'Uncategorized'].tail(50)
#df_k_uni['job_title'][df_k_uni['job_category'] == 'Too vague answers'].head(50)

Optional: Check how these categories got populated actually

In [ ]:
#for category in job_categories:
#    unique_titles = df_it_uni['job_title'][df_it_uni['job_category'] == category].unique()
#    print(f"{category}: {unique_titles}")

In [ ]:
#for category in job_categories:
#    unique_titles = df_ai_uni['job_title'][df_ai_uni['job_category'] == category].unique()
#    print(f"{category}: {unique_titles}")

In [ ]:
#for category in job_categories:
#    unique_titles = df_k_uni['job_title'][df_k_uni['job_category'] == category].unique()
#    print(f"{category}: {unique_titles}")

## Western Countries

In [ ]:
western_countries = [
    'al', 'ad', 'am', 'at', 'az', 'by', 'be', 'ba', 'bg', 'hr',
    'cy', 'cz', 'dk', 'ee', 'fi', 'fr', 'ge', 'de', 'gr', 'hu',
    'is', 'ie', 'it', 'kz', 'xk', 'lv', 'li', 'lt', 'lu', 'mt',
    'md', 'mc', 'me', 'nl', 'mk', 'no', 'pl', 'pt', 'ro', 'ru',
    'sm', 'rs', 'sk', 'si', 'es', 'se', 'ch', 'tr', 'ua', 'gb',
    'va', 'ca', 'au', 'us'
]

In [ ]:
developed_countries = ['de', 'gb', 'nl', 'se', 'dk', 'be', 'fi', 'at', 'ch', 'ie', 'ca', 'au', 'us']

## Log-transformed salary

The reasonability of this step was assessed later, in the Analysis part, and iteratively added back to here.

In [ ]:
dataframes = [df_k_uni, df_it_uni, df_ai_uni]
columns_to_transform = ['salary', 'salary_2024', 'salary_norm', 'salary_norm_2024', 'salary_normmed_2024', 'salary_normmean_2024', 'salary_normgdp_2024', 'salary_normse_2024']

for df in dataframes:
    for col in columns_to_transform:
        df[f'{col}_log'] = np.log(df[col])

<div class="c">
Log-transformation was later discovered to be a not just a <u>practical</u> but also a <b>necessary</b> step.<br>
But of course, this realization was born later, and the transformation step inserted here to give the project a more logically followable structure.
</div> 

# Outlier detection

<div class="c">
To detect outliers, besides Z-score, the literature often advises the use of modified Z-score, which is basically its nonparameteric counterpart, operating with medians instead of means.<br>
Furthermore, a signed-modified-Z-score is also calculated, as the modified-Z-score loses the sign-dependence (will not distinguish between outliers that are too-low or too-high). <br>
Another popular approach can be the 1.5 IQR interval. Using Z-scores was a design choice.
</div> 

In [ ]:
def add_z_scores(data, numerical_column):
    # Calculate Z-scores
    mean = data[numerical_column].mean()
    std = data[numerical_column].std()
    z_scores = (data[numerical_column] - mean) / std
    data['z_score'] = z_scores
    
    # Calculate modified Z-scores
    # 0.6745 * (xi - median)/MAD, where xi is the actual row that the Z-score will be calculated to. MAD is the median absolute deviation.
    median = data[numerical_column].median()
    mad = np.median(np.abs(data[numerical_column] - median))
    modif_z_scores = 0.6745 * np.abs(data[numerical_column] - median) / mad
    data['modif_z_score'] = modif_z_scores

    # Calculate modified Z-scores-signed
    modif_z_scores_signed = 0.6745 * (data[numerical_column] - median) / mad
    data['modif_z_score_signed'] = modif_z_scores_signed
    
    return data

In [ ]:
def detect_outliers(data):
    threshold = 10  # Adjust threshold as needed
    median = data.median()
    median_absolute_deviation = np.median(np.abs(data - median))
    modified_z_scores = 0.6745 * np.abs(data - median) / median_absolute_deviation
    return modified_z_scores > threshold

In [ ]:
def detect_outliers_1006_2035(data, numerical_column, lower_threshold=-3, upper_threshold=10):
    median = data[numerical_column].median()
    mad = np.median(np.abs(data[numerical_column] - median))
    modif_z_scores_signed = 0.6745 * (data[numerical_column] - median) / mad
    outliers = data[(modif_z_scores_signed < lower_threshold) | (modif_z_scores_signed > upper_threshold)]
    return outliers

## Based on Normalized, Log-transformed salary

### Kaggle

In [ ]:
df_k_uni_copy = df_k_uni.copy()
df_ai_uni_copy = df_ai_uni.copy()
df_it_uni_copy = df_it_uni.copy()

In [ ]:
# grouped DF
df_k_g = df_k_uni.groupby(['experience'])

# adding Z-scores
df_k_gz = df_k_g.apply(lambda x: add_z_scores(x, 'salary_normmed_2024_log'), include_groups=False).reset_index(drop=True)

# outputting with descending Modified-Z-score order
df_k_gz.sort_values('modif_z_score_signed', ascending=True).head(5)

In [ ]:
# Sorting the dataframes for plotting
df_k_gzs = df_k_gz['z_score'].sort_values(ascending=False).reset_index(drop=True)
df_k_gmzs = df_k_gz['modif_z_score'].sort_values(ascending=False).reset_index(drop=True)
df_k_gmzs_signed = df_k_gz['modif_z_score_signed'].sort_values(ascending=False).reset_index(drop=True)

# Plotting
plt.figure(figsize=(20, 5))

plt.subplot(1, 3, 1)
plt.plot(df_k_gzs, marker='.', linestyle='-', color='b')
plt.title('Sorted Z-scores')
plt.xlabel('Index')
plt.ylabel('Z-score')

plt.subplot(1, 3, 2)
plt.plot(df_k_gmzs, marker='.', linestyle='-', color='r')
plt.title('Sorted Modified Z-scores (Absolute)')
plt.xlabel('Index')
plt.ylabel('Modified Z-score')

plt.subplot(1, 3, 3)
plt.plot(df_k_gmzs_signed, marker='.', linestyle='-', color='g')
plt.title('Sorted Modified Z-scores (Signed)')
plt.xlabel('Index')
plt.ylabel('Modified Z-score (Signed)')

plt.tight_layout()
plt.show()

In [ ]:
# Start with the initial DataFrame
len_k_initial = len(df_k_gz)

# Define thresholds
lower_threshold = -0.5
upper_threshold = 3.0

# Boolean masks for outliers
mask_k_small = df_k_gz['modif_z_score_signed'] < lower_threshold
mask_k_large = df_k_gz['modif_z_score_signed'] > upper_threshold

# Count outliers
len_k_outlierdrop_small = mask_k_small.sum()
len_k_outlierdrop_large = mask_k_large.sum()

# Remove outliers
df_k = df_k_gz[~(mask_k_small | mask_k_large)].copy()

# Print results
print(f"{len_k_outlierdrop_small + len_k_outlierdrop_large} outliers removed out of {len_k_initial} rows:")
print(f" - Too small: {len_k_outlierdrop_small} rows removed")
print(f" - Too large: {len_k_outlierdrop_large} rows removed")

# Display the top rows sorted by 'modif_z_score'
df_k.sort_values('modif_z_score', ascending=False).head(2)

In [ ]:
df_k[df_k['country'].isin(developed_countries)].sort_values('modif_z_score_signed', ascending=True).head(10)

### DE-IT

In [ ]:
# grouped DF
df_it_g = df_it_uni.groupby(['seniority_level'])

# adding Z-scores
df_it_gz = df_it_g.apply(lambda x: add_z_scores(x, 'salary_normmed_2024_log'), include_groups=False).reset_index()

# outputting with descending Modified-Z-score order
df_it_gz.sort_values('modif_z_score', ascending=False).head()

In [ ]:
# Sorting the dataframes for plotting
df_it_gzs = df_it_gz['z_score'].sort_values(ascending=False).reset_index(drop=True)
df_it_gmzs = df_it_gz['modif_z_score'].sort_values(ascending=False).reset_index(drop=True)
df_it_gmzs_signed = df_it_gz['modif_z_score_signed'].sort_values(ascending=False).reset_index(drop=True)

# Plotting
plt.figure(figsize=(20, 5))

plt.subplot(1, 3, 1)
plt.plot(df_it_gzs, marker='.', linestyle='-', color='b')
plt.title('Sorted Z-scores')
plt.xlabel('Index')
plt.ylabel('Z-score')

plt.subplot(1, 3, 2)
plt.plot(df_it_gmzs, marker='.', linestyle='-', color='r')
plt.title('Sorted Modified Z-scores (Absolute)')
plt.xlabel('Index')
plt.ylabel('Modified Z-score')

plt.subplot(1, 3, 3)
plt.plot(df_it_gmzs_signed, marker='.', linestyle='-', color='g')
plt.title('Sorted Modified Z-scores (Signed)')
plt.xlabel('Index')
plt.ylabel('Modified Z-score (Signed)')

plt.tight_layout()
plt.show()

In [ ]:
# Start with the initial DataFrame
len_it_initial = len(df_it_gz)

# Define thresholds
lower_threshold = -3.0
upper_threshold = 3.0

# Boolean masks for outliers
mask_it_small = df_it_gz['modif_z_score_signed'] < lower_threshold
mask_it_large = df_it_gz['modif_z_score_signed'] > upper_threshold

# Count outliers
len_it_outlierdrop_small = mask_it_small.sum()
len_it_outlierdrop_large = mask_it_large.sum()

# Remove outliers
df_it = df_it_gz[~(mask_it_small | mask_it_large)].copy()

# Print results
print(f"{len_it_outlierdrop_small + len_it_outlierdrop_large} outliers removed out of {len_it_initial} rows:")
print(f" - Too small: {len_it_outlierdrop_small} rows removed")
print(f" - Too large: {len_it_outlierdrop_large} rows removed")

# Display the top 2 rows sorted by 'modif_z_score'
df_it.sort_values('modif_z_score', ascending=False).head(2)

### AI-Jobs.net

In [ ]:
# grouped DF
df_ai_g = df_ai_uni.groupby(['seniority_level'], observed=False)

# adding Z-scores
df_ai_gz = df_ai_g.apply(lambda x: add_z_scores(x, 'salary_normmed_2024_log'), include_groups=False).reset_index()

# outputting with descending Modified-Z-score order
df_ai_gz.sort_values('modif_z_score_signed', ascending=True).head()

In [ ]:
# Sorting the dataframes for plotting
df_ai_gzs = df_ai_gz['z_score'].sort_values(ascending=False).reset_index(drop=True)
df_ai_gmzs = df_ai_gz['modif_z_score'].sort_values(ascending=False).reset_index(drop=True)
df_ai_gmzs_signed = df_ai_gz['modif_z_score_signed'].sort_values(ascending=False).reset_index(drop=True)

# Plotting
plt.figure(figsize=(20, 5))

plt.subplot(1, 3, 1)
plt.plot(df_ai_gzs, marker='.', linestyle='-', color='b')
plt.title('Sorted Z-scores')
plt.xlabel('Index')
plt.ylabel('Z-score')

plt.subplot(1, 3, 2)
plt.plot(df_ai_gmzs, marker='.', linestyle='-', color='r')
plt.title('Sorted Modified Z-scores (Absolute)')
plt.xlabel('Index')
plt.ylabel('Modified Z-score')

plt.subplot(1, 3, 3)
plt.plot(df_ai_gmzs_signed, marker='.', linestyle='-', color='g')
plt.title('Sorted Modified Z-scores (Signed)')
plt.xlabel('Index')
plt.ylabel('Modified Z-score (Signed)')

plt.tight_layout()
plt.show()

In [ ]:
# Start with the initial DataFrame
len_ai_initial = len(df_ai_gz)

# Define thresholds
lower_threshold = -3.0
upper_threshold = 3.0

# Boolean masks for outliers
mask_ai_small = df_ai_gz['modif_z_score_signed'] < lower_threshold
mask_ai_large = df_ai_gz['modif_z_score_signed'] > upper_threshold

# Count outliers
len_ai_outlierdrop_small = mask_ai_small.sum()
len_ai_outlierdrop_large = mask_ai_large.sum()

# Remove outliers
df_ai = df_ai_gz[~(mask_ai_small | mask_ai_large)].copy()

# Print results
print(f"{len_ai_outlierdrop_small + len_ai_outlierdrop_large} outliers removed out of {len_ai_initial} rows:")
print(f" - Too small: {len_ai_outlierdrop_small} rows removed")
print(f" - Too large: {len_ai_outlierdrop_large} rows removed")

# Display the top 2 rows sorted by 'modif_z_score'
df_ai.sort_values('modif_z_score', ascending=False).head(2)

#### Final check 

In [ ]:
df_k[df_k['country'].isin(developed_countries)].sort_values('modif_z_score_signed', ascending=True).head(20)

In [ ]:
df_ai.sort_values('modif_z_score', ascending=False).head(5)

In [ ]:
df_it.sort_values('modif_z_score', ascending=False).head(5)

<div class="c">
Conclusion:<br>
The commonly used modified-Z-score loses the sign-dependence, which is a crucial information here (so it will not distinguish between a too-low and a too-high salary).<br>
Sign dependence gives significant information here, as no matter log-transformation or not, the distribution of salaries will always be closed in one-end (the lower end) and open by the higher end.<br>
Manual inspection shows that with a symmetric modif-Z-score cutoff, the irrealistically low answers are not dropped. <br>
This is most notable for the Kaggle surveys:<br> As the survey did not clearly specify whether the salary question is asking a yearly or a monthly value! This is an trivial yet enormous error in survey design<br>
<br>
To counteract this, I used a strict cutoff limit for the lower outlier for Kaggle. This was done iteratively paired with careful manual inspection.
</div> 

# Data Quality Metrics

<div class="c">
Through the cleaning steps, several 'len_' variables were introduced which tracked the length of each survey as the cleaning step was applied.<br>
In this chapter, these counters are collected.<br>
<br>
This is not done quite elegantly, but this ugly approach enables enormous flexibility in reorganizing the cleaning steps, and injecting new steps.
</div> 

## Germany-It Survey

In [ ]:
cleaning_steps = [
    'Salary-nulls',
    'Employment-nulls',
    'Students',  
    'Never have coded',
    'Country-nulls',
    'Seniority-nulls',
    'Job-title-nulls',
    'Outliers: Too small salary',
    'Outliers: Too large salary'
]

In [ ]:
len_it_ini = sum([
    len_it18_ini,
    len_it19_ini,
    len_it20_ini,
    len_it21_ini,
    len_it22_ini,
    len_it23_ini
])

len_it18_salarydrop = len_it18_salarydrop1 - len_it18_salarydrop2
len_it19_salarydrop = len_it19_salarydrop1 - len_it19_salarydrop2
len_it20_salarydrop = len_it20_salarydrop1 - len_it20_salarydrop2
len_it21_salarydrop = len_it21_salarydrop1 - len_it21_salarydrop2
len_it22_salarydrop = len_it22_salarydrop1 - len_it22_salarydrop2
len_it23_salarydrop = len_it23_salarydrop1 - len_it23_salarydrop2

len_it_salarydrop = sum([
    len_it18_salarydrop,
    len_it19_salarydrop,
    len_it20_salarydrop,
    len_it21_salarydrop,
    len_it22_salarydrop,
    len_it23_salarydrop
])

len_it_employmentdrop  = len_it_employmentdrop1  - len_it_employmentdrop2
len_it_studentdrop     = len_it_studentdrop1     - len_it_studentdrop2
len_it_noncoderdrop    = 0
len_it_countrydrop     = len_it_countrydrop1     - len_it_countrydrop2
len_it_senioritydrop   = len_it_senioritydrop1   - len_it_senioritydrop2
len_it_jobtitledrop    = len_it_jobtitledrop1    - len_it_jobtitledrop2
#len_it_outlierdrop     = len_it_outlierdrop1     - len_it_outlierdrop2
#len_it_outlierdropnorm = len_it_outlierdropnorm1 - len_it_outlierdropnorm2

len_it_clean = (len(df_it))

it_difference  = len_it_ini - len_it_clean
it_cleanedaway = [
    len_it_salarydrop,
    len_it_employmentdrop,
    len_it_studentdrop,   
    len_it_noncoderdrop,
    len_it_countrydrop,
    len_it_senioritydrop,
    len_it_jobtitledrop,
    len_it_outlierdrop_small,
    len_it_outlierdrop_large
]

In [ ]:
print(f'Initial survey length: {len_it_ini}')
# Printing each variable in the list
for idx, value in enumerate(it_cleanedaway):
    print(f'Cleaning step {idx + 1}: {value}')
    
print(f'Final survey length: {len_it_clean}')
print(f'The difference between final and initial: {it_difference}')
print(f'Summing the individual cleaning steps: {sum(it_cleanedaway)}')

## Kaggle

In [ ]:
len_k_ini = sum([
    len_k19_ini,
    len_k20_ini,
    len_k21_ini,
    len_k22_ini
])

len_k19_salarydrop = len_k19_salarydrop1 - len_k19_salarydrop2
len_k20_salarydrop = len_k20_salarydrop1 - len_k20_salarydrop2
len_k21_salarydrop = len_k21_salarydrop1 - len_k21_salarydrop2
len_k22_salarydrop = len_k22_salarydrop1 - len_k22_salarydrop2

len_k_salarydrop = sum([
    len_k19_salarydrop,
    len_k20_salarydrop,
    len_k21_salarydrop,
    len_k22_salarydrop
])

len_k_employmentdrop  = 0
len_k_studentdrop     = 0
len_k_noncoderdrop    = len_k_noncoderdrop1    - len_k_noncoderdrop2
len_k_countrydrop     = len_k_countrydrop1     - len_k_countrydrop2
len_k_senioritydrop   = len_k_senioritydrop1   - len_k_senioritydrop2
len_k_jobtitledrop    = len_k_jobtitledrop1    - len_k_jobtitledrop2
#len_k_outlierdrop     = len_k_outlierdrop1     - len_k_outlierdrop2
#len_k_outlierdropnorm = len_k_outlierdropnorm1 - len_k_outlierdropnorm2

len_k_clean = (len(df_k))

k_difference  = len_k_ini - len_k_clean
k_cleanedaway = [
    len_k_salarydrop,
    len_k_employmentdrop,
    len_k_studentdrop,   
    len_k_noncoderdrop,
    len_k_countrydrop,
    len_k_senioritydrop,
    len_k_jobtitledrop,
    len_k_outlierdrop_small,
    len_k_outlierdrop_large
]

k_salarydrops = [
    len_k19_salarydrop,
    len_k20_salarydrop,
    len_k21_salarydrop,
    len_k22_salarydrop
]

In [ ]:
print(f'Initial survey length: {len_k_ini}')
# Printing each variable in the list
for idx, value in enumerate(k_cleanedaway):
    print(f'Cleaning step {idx + 1}: {value}')
    
print(f'Final survey length: {len_k_clean}')
print(f'The difference between final and initial: {k_difference}')
print(f'Summing the individual cleaning steps: {sum(k_cleanedaway)}')

## AI-Jobs.net

In [ ]:
#len_ai_ini
len_ai_salarydrop = 0

len_ai_employmentdrop  = len_ai_employmentdrop1  - len_ai_employmentdrop2
len_ai_studentdrop     = len_ai_studentdrop1     - len_ai_studentdrop2
len_ai_noncoderdrop    = 0
len_ai_countrydrop     = len_ai_countrydrop1     - len_ai_countrydrop2
len_ai_senioritydrop   = len_ai_senioritydrop1   - len_ai_senioritydrop2
len_ai_jobtitledrop    = len_ai_jobtitledrop1    - len_ai_jobtitledrop2
#len_ai_outlierdrop     = len_ai_outlierdrop1     - len_ai_outlierdrop2
#len_ai_outlierdropnorm = len_ai_outlierdropnorm1 - len_ai_outlierdropnorm2

len_ai_clean = (len(df_ai))

ai_difference  = len_ai_ini - len_ai_clean
ai_cleanedaway = [
    len_ai_salarydrop,
    len_ai_employmentdrop,
    len_ai_studentdrop,   
    len_ai_noncoderdrop,
    len_ai_countrydrop,
    len_ai_senioritydrop,
    len_ai_jobtitledrop,
    len_ai_outlierdrop_small,
    len_ai_outlierdrop_large
]

In [ ]:
print(f'Initial survey length: {len_ai_ini}')
# Printing each variable in the list
for idx, value in enumerate(ai_cleanedaway):
    print(f'Cleaning step {idx + 1}: {value}')
    
print(f'Final survey length: {len_ai_clean}')
print(f'The difference between final and initial: {ai_difference}')
print(f'Summing the individual cleaning steps: {sum(ai_cleanedaway)}')

## Plots

In [ ]:
# Data to plot
labels = cleaning_steps
sizes = it_cleanedaway

# Filter out zero segments
non_zero_indices = [i for i, size in enumerate(sizes) if size > 0]
filtered_labels = [labels[i] for i in non_zero_indices]
filtered_sizes = [sizes[i] for i in non_zero_indices]

# Create an explode list to separate slices for better readability
explode = [0.1] * len(filtered_sizes)  # Explode all slices for visibility

# Create the pie chart
plt.figure(figsize=(4, 4))
wedges, texts, autotexts = plt.pie(
    filtered_sizes,
    #explode=explode,
    labels=filtered_labels,
    autopct='%1.1f%%',
    #shadow=True,
    startangle=140,
    pctdistance=0.8,  # Distance of percentage from the center
    labeldistance=1.1,  # Distance of labels from the center
    wedgeprops=dict(width=0.5, edgecolor='white', linewidth=2)  # This creates the ring effect
)

# Formatting labels and percentages
for text in texts:
    text.set_fontsize(12)
for autotext in autotexts:
    autotext.set_fontsize(10)
    autotext.set_color('white')

plt.title('Germany-IT survey: ratio of cleaned-away data')
plt.show()


In [ ]:
# Data to plot
labels = cleaning_steps
sizes = ai_cleanedaway

# Filter out zero segments
non_zero_indices = [i for i, size in enumerate(sizes) if size > 0]
filtered_labels = [labels[i] for i in non_zero_indices]
filtered_sizes = [sizes[i] for i in non_zero_indices]

# Create an explode list to separate slices for better readability
explode = [0.1] * len(filtered_sizes)  # Explode all slices for visibility

# Create the pie chart
plt.figure(figsize=(4, 4))
wedges, texts, autotexts = plt.pie(
    filtered_sizes,
    #explode=explode,
    labels=filtered_labels,
    autopct='%1.1f%%',
    #shadow=True,
    startangle=140,
    pctdistance=0.8,  # Distance of percentage from the center
    labeldistance=1.1,
    wedgeprops=dict(width=0.5, edgecolor='white', linewidth=2) 
)

# Formatting labels and percentages
for text in texts:
    text.set_fontsize(12)
for autotext in autotexts:
    autotext.set_fontsize(10)
    autotext.set_color('white')

plt.title('AI-Jobs.net: ratio of cleaned-away data')
plt.show()

In [ ]:
# Data to plot
labels = cleaning_steps
sizes = k_cleanedaway

# Filter out zero segments
non_zero_indices = [i for i, size in enumerate(sizes) if size > 0]
filtered_labels = [labels[i] for i in non_zero_indices]
filtered_sizes = [sizes[i] for i in non_zero_indices]

# Create an explode list to separate slices for better readability
explode = [0.1] * len(filtered_sizes)  # Explode all slices for visibility

# Create the pie chart
plt.figure(figsize=(4, 4))
wedges, texts, autotexts = plt.pie(
    filtered_sizes,
    #explode=explode,
    labels=filtered_labels,
    autopct='%1.1f%%',
    #shadow=True,
    startangle=45,
    pctdistance=0.8,  # Distance of percentage from the center
    labeldistance=1.1,  # Distance of labels from the center
    wedgeprops=dict(width=0.5, edgecolor='white', linewidth=2) 
)

# Formatting labels and percentages
for text in texts:
    text.set_fontsize(12)
for autotext in autotexts:
    autotext.set_fontsize(10)
    autotext.set_color('white')

plt.title('Kaggle: ratio of cleaned-away data')
plt.show()

In [ ]:
cleaning_steps

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import numpy as np

# Assuming your data is defined as follows:
# cleaning_steps = ['Step1', 'Step2', 'Step3', 'Step4', 'Step5', 'Step6', 'Step7']
# it_cleanedaway = [...]
# ai_cleanedaway = [...]
# k_cleanedaway = [...]

labels = cleaning_steps
sizes_it = it_cleanedaway
sizes_ai = ai_cleanedaway
sizes_k = k_cleanedaway

# Define manual colors for each cleaning step
step_colors = {
    'Salary-nulls': '#161359',           # Blue
    'Employment-nulls': '#2e4263',       # Orange
    'Students': '#4095c9',               # Green
    'Never have coded': '#5ebdba',       # Red
    'Country-nulls': '#edc600',          # Purple
    'Seniority-nulls': '#e0d180',        # Brown
    'Job-title-nulls': '#a19387',        # Pink
    'Outliers: Too small salary': '#b0350c',  # Cyan
    'Outliers: Too large salary': '#c77f7f'   # Olive
}

def filter_data(labels, sizes):
    non_zero_indices = [i for i, size in enumerate(sizes) if size > 0]
    filtered_labels = [labels[i] for i in non_zero_indices]
    filtered_sizes = [sizes[i] for i in non_zero_indices]
    return filtered_labels, filtered_sizes

# Filter data for each dataframe
labels_it, sizes_it = filter_data(labels, sizes_it)
labels_ai, sizes_ai = filter_data(labels, sizes_ai)
labels_k, sizes_k = filter_data(labels, sizes_k)

fig, axes = plt.subplots(1, 3, figsize=(8, 5))

# Function to plot pie charts
def plot_pie(ax, sizes, labels, title):
    colors_list = [step_colors[label] for label in labels]
    wedges, _ = ax.pie(
        sizes,
        colors=colors_list,
        startangle=140,
        wedgeprops=dict(width=0.5, edgecolor='white', linewidth=2)
    )
    ax.set_title(title)
    # Add percentages next to the pie
    total = sum(sizes)
    angles = [wedge.theta2 - (wedge.theta2 - wedge.theta1) / 2. for wedge in wedges]
    for i, angle in enumerate(angles):
        x = np.cos(np.deg2rad(angle))
        y = np.sin(np.deg2rad(angle))
        ax.text(1.2 * x, 1.2 * y, f'{(sizes[i] / total) * 100:.1f}%', ha='center', va='center', fontsize=10)

# Plot for IT Cleaned Away
plot_pie(axes[0], sizes_it, labels_it, 'Germany-IT')

# Plot for AI Cleaned Away
plot_pie(axes[1], sizes_ai, labels_ai, 'AI-Jobs.net')

# Plot for K Cleaned Away
plot_pie(axes[2], sizes_k, labels_k, 'Kaggle')

# Create a vertical legend
legend_elements = [Patch(facecolor=step_colors[label], label=label) for label in labels]
fig.legend(
    handles=legend_elements, 
    loc='center right', 
    bbox_to_anchor=(1.15, 0.5), 
    ncol=1, 
    title="Cleaning Steps"
)

# Adjust layout to make space for legend
plt.tight_layout(rect=[0, 0, 0.9, 1])  # Leave space on the right for the legend

# Set the overall title
fig.suptitle('Ratio of Cleaned-Away Data for IT, AI, and K', fontsize=16)

plt.show()

In [ ]:
# Data to plot
labels = [2019,2020,2021,2022]
sizes = k_salarydrops

# Filter out zero segments
non_zero_indices = [i for i, size in enumerate(sizes) if size > 0]
filtered_labels = [labels[i] for i in non_zero_indices]
filtered_sizes = [sizes[i] for i in non_zero_indices]

# Create an explode list to separate slices for better readability
explode = [0.1] * len(filtered_sizes)  # Explode all slices for visibility

# Create the pie chart
plt.figure(figsize=(4, 4))
wedges, texts, autotexts = plt.pie(
    filtered_sizes,
    #explode=explode,
    labels=filtered_labels,
    autopct='%1.1f%%',
    #shadow=True,
    startangle=140,
    pctdistance=0.8,  # Distance of percentage from the center
    labeldistance=1.1,  # Distance of labels from the center
    wedgeprops=dict(width=0.5, edgecolor='white', linewidth=2) 
)

# Formatting labels and percentages
for text in texts:
    text.set_fontsize(12)
for autotext in autotexts:
    autotext.set_fontsize(10)
    autotext.set_color('white')

plt.title('IT Industry Drops')
plt.show()

In [ ]:
# Data
categories = ['Germany IT-Survey', 'Kaggle', 'Ai-Jobs']
data_points_1 = [len_it_clean, sum(it_cleanedaway)]
data_points_2 = [len_k_clean, sum(k_cleanedaway)]
data_points_3 = [len_ai_clean, sum(ai_cleanedaway)]

# Create the figure and axis
fig, ax = plt.subplots(figsize=(6, 3))
bar_width = 0.35  # Width of the bars

# Colors for the stacks
colors = ['green', 'red']
colors = ['darkgreen', 'firebrick']
colors = ['olive', 'maroon']
colors = ['forestgreen', 'crimson']
colors = ['seagreen', 'darkred']
colors = ['teal', 'darkred']
colors = ['seagreen', 'maroon']

# Plot each bar category
bottom = [0] * len(categories)
for i, category in enumerate(categories):
    for j, value in enumerate([data_points_1, data_points_2, data_points_3][i]):
        ax.bar(category, value, bar_width, bottom=bottom[i], color=colors[j])
        bottom[i] += value

# Adding labels and title
ax.set_ylabel('Responses')
ax.set_title('Discarded Responses from the Surveys')

# Create custom legend
handles = [plt.Rectangle((0,0),1,1, color=color) for color in colors]
labels = ['Remaining Data', 'Cleaned-away Data']
ax.legend(handles, labels, loc='upper right')

# Show plot
plt.show()

In [ ]:
# Create the figure and axis with specified size
fig, ax = plt.subplots(figsize=(4.5, 3))  # Set the figure size here
bar_width = 0.35  # Width of the bars

# Colors for the stacks
colors = ['seagreen', 'maroon']

# Plot each bar category
bottom = [0] * len(categories)
for i, category in enumerate(categories):
    for j, value in enumerate([data_points_1, data_points_2, data_points_3][i]):
        ax.bar(category, value, bar_width, bottom=bottom[i], color=colors[j])
        bottom[i] += value

# Adding labels and title
ax.set_ylabel('Responses')
ax.set_title('Discarded Responses from the Surveys')

# Create custom legend
handles = [plt.Rectangle((0, 0), 1, 1, color=color) for color in colors]
labels = ['Remaining Data', 'Cleaned-away Data']
ax.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=2)

# Adjust layout to fit legend outside the plot
plt.tight_layout()

# Show plot
plt.show()

<div class="c">
Conclusion:<br>
There is a significant disparity in the data quality between the three main sources of surveys.<br>
This can be expected, as Kaggle asked a tremendous amount of questions, and by its nature is targeting the masses.<br>
In contrast, Germany-IT survey seems to have been distributed more personally (or let's say it is not tied to an international social-network).<br>
About AI-Jobs.net's methodology, we do not know much. It is surprisingly clean.
</div> 

# Exporting the cleaned data

In [ ]:
import os

# Dictionary of DataFrames and their corresponding filenames
dataframes = {
    'df_k': df_k,
    'df_it': df_it,
    'df_ai': df_ai
    # Add more DataFrames and filenames as needed
}
# Path to the cleaned data folder
cleaned_data_folder = '../data/cleaned/'

# Ensure the folder exists
os.makedirs(cleaned_data_folder, exist_ok=True)

# Loop through the dictionary and export each DataFrame to a CSV file
for filename, dataframe in dataframes.items():
    full_path = os.path.join(cleaned_data_folder, f'{filename}.csv')
    dataframe.to_csv(full_path, index=False)

print(f'Individual DataFrames and combined DataFrame exported to {cleaned_data_folder}')

In [ ]:
# Combine DataFrames into one
df_combined = pd.concat(dataframes.values(), ignore_index=True)

# Path to the cleaned data folder
cleaned_data_folder = '../data/cleaned/'

# Ensure the folder exists
os.makedirs(cleaned_data_folder, exist_ok=True)

# Export the combined DataFrame to CSV
combined_path = os.path.join(cleaned_data_folder, 'df_combined_tableau.csv')
df_combined.to_csv(combined_path, index=False, encoding='utf-8')

print(f'Individual DataFrames and combined DataFrame exported to {cleaned_data_folder}')